In [1]:
from collections import Counter, defaultdict
import re
import time
from typing import List, Literal, Optional
from concurrent.futures import ThreadPoolExecutor, as_completed
import random
import string
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pydantic import BaseModel, Field

from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer
import torch

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

from bertopic import BERTopic
from bertopic.representation import TextGeneration
from umap import UMAP
from hdbscan import HDBSCAN

from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.cluster import DBSCAN

import boe_risk_monitoring.config as config
from boe_risk_monitoring.llms.processing_llms import TopicLabellingLLM, QuestionAnswerTaggingLLM, EvasivenessTaggingLLM, SentimentAnalysisLLM, RiskLabellingLLM

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rahim1z\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rahim1z\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rahim1z\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\rahim1z\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
# Check if CUDA is available and print the version
print(torch.version.cuda)
print(torch.cuda.is_available())

11.8
True


#### **SET GLOBAL VARIABLES**

In [3]:
# Make sure the working directory is set correctly
print("Initial working directory: ", os.getcwd())
os.chdir("../..")
print("New working directory: ", os.getcwd())

Initial working directory:  c:\python\bank-of-eng-risk-monitoring\boe_risk_monitoring\nlp
New working directory:  c:\python\bank-of-eng-risk-monitoring


In [52]:
ROOT_FPATH = os.getcwd()
DATA_FOLDER_PATH = config.DATA_FOLDER_PATH
AGGREGATED_DATA_FOLDER_PATH = config.AGGREGATED_DATA_FOLDER_PATH
ALL_TEXT_FNAME = "all_text.parquet"
ALL_TEXT_FPATH = os.path.join(AGGREGATED_DATA_FOLDER_PATH, ALL_TEXT_FNAME)
GLOSSARY_FNAME = "glossary_dictionary_citigroup.csv"
GLOSSARY_FPATH = os.path.join(DATA_FOLDER_PATH, GLOSSARY_FNAME)
APP_DATA_FOLDER_PATH = os.path.join(DATA_FOLDER_PATH, "app")

RERUN_SENTIMENT_ANALYSIS = False
RERUN_UNSUPERVISED_TOPIC_MODELLING = False
RERUN_GUIDED_TOPIC_MODELLING = False
RERUN_ZERO_SHOT_TOPIC_MODELLING = False
RERUN_RISK_CATEGORY_LABELLING = False
RUN_RISK_CATEGORY_SCORING = False
RERUN_MULTI_TOPIC_LABELLING = False
RERUN_Q_AND_A_TAGGING = False
RERUN_EVASIVE_ANSWER_DETECTION = True

In [5]:
# Load the glossary dictionary
GLOSSARY_DF = pd.read_csv(GLOSSARY_FPATH)
GLOSSARY_DICT = GLOSSARY_DF.set_index('Term')['Definition'].to_dict()

#### **SETUP**

In [6]:
# Read in the data
df_all_text = pd.read_parquet(ALL_TEXT_FPATH)
df_all_text

,text,fiscal_period_ref,source,role,page,section,reporting_period,publication_date,bank,document_type,reference,source_type,is_comparative
0,"Hello, and welcome to Citi's First Quarter 202...",quarter,Operator,Host,1.0,Introduction,Q1_2023,2023-04-14,Citigroup,transcript,"Operator (Host)\nCitigroup, Q1, 2023 Earnings ...",internal,False
1,"Ms. Landis, you may begin.",quarter,Operator,Host,1.0,Introduction,Q1_2023,2023-04-14,Citigroup,transcript,"Operator (Host)\nCitigroup, Q1, 2023 Earnings ...",internal,False
2,"Thank you, operator. Good morning and thank yo...",quarter,Jennifer Landis,Host,1.0,Disclaimer,Q1_2023,2023-04-14,Citigroup,transcript,"Jennifer Landis (Host)\nCitigroup, Q1, 2023 Ea...",internal,False
3,"With that, I'll turn it over to Jane.",quarter,Jennifer Landis,Host,1.0,Introduction,Q1_2023,2023-04-14,Citigroup,transcript,"Jennifer Landis (Host)\nCitigroup, Q1, 2023 Ea...",internal,False
4,"Thank you, Jenn, hello to everyone joining us ...",quarter,Jane Fraser,CEO,1.0,Prepared remarks,Q1_2023,2023-04-14,Citigroup,transcript,"Jane Fraser (CEO)\nCitigroup, Q1, 2023 Earning...",internal,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16525,Earnings gap between trading and investment ba...,,Joshua Franklin,,,,Q1_2025,2025-04-08,JPMorgan,news,News article by Joshua Franklin\nTitle: Wall S...,external,True
16526,Profits at large US banks set to rise year on ...,,Joshua Franklin,,,,Q1_2025,2025-04-08,Citigroup,news,News article by Joshua Franklin\nTitle: Wall S...,external,True
16527,Profits at large US banks set to rise year on ...,,Joshua Franklin,,,,Q1_2025,2025-04-08,Bank of America,news,News article by Joshua Franklin\nTitle: Wall S...,external,True
16528,Profits at large US banks set to rise year on ...,,Joshua Franklin,,,,Q1_2025,2025-04-08,JPMorgan,news,News article by Joshua Franklin\nTitle: Wall S...,external,True


In [7]:
# Let's process the reporting period to give something which sorts chronologically
reporting_period_split_srs = df_all_text['reporting_period'].str.split("_")
df_all_text['reporting_period'] = reporting_period_split_srs.str.get(1).astype(str) + reporting_period_split_srs.str.get(0)
df_all_text['reporting_period'].value_counts()

reporting_period
2023Q1    2195
2024Q3    1999
2023Q4    1993
2023Q2    1947
2025Q1    1817
2024Q2    1752
2024Q1    1751
2023Q3    1655
2024Q4    1421
Name: count, dtype: int64

In [8]:
# Check categories of text
df_all_text['section'].value_counts(dropna=False, normalize=True).head(20)

section
Financial Results    0.483606
Q and A              0.232244
Prepared remarks     0.173624
Footnotes            0.042468
                     0.025045
Outlook              0.011736
Glossary             0.008167
Vision               0.007139
Conclusion           0.005808
Introduction         0.005505
Disclaimer           0.003751
Title                0.000907
Name: proportion, dtype: float64

In [9]:
# Drop text tagged as Introduction, Conclusion, Disclaimer or Title
df_all_text_main = df_all_text[~df_all_text['section'].isin(['Introduction', 'Conclusion', 'Disclaimer', 'Title'])].reset_index(drop=True)

In [10]:
# We'll also drop text from the Operator
df_all_text_main['source'].value_counts().head(20)

source
Jeremy Barnum            1244
Alastair Borthwick       1062
Brian Moynihan            952
Mark Mason                924
Jane Fraser               544
Operator                  276
Jamie Dimon               255
MARK MASON                179
Mike Mayo                 139
JANE FRASER               127
Gerard Cassidy             99
Erika Najarian             94
Stephen Gandel             83
Alastair M. Borthwick      82
Glenn Schorr               75
Jim Mitchell               74
Joshua Franklin            65
Steven Chubak              60
John McDonald              58
Ken Usdin                  56
Name: count, dtype: int64

In [11]:
df_all_text_main = df_all_text_main[~df_all_text_main['source'].isin(['Operator'])].reset_index(drop=True)

In [12]:
# Print rows before and after dropping
print("Rows before dropping:", df_all_text.shape[0])
print("Rows after dropping:", df_all_text_main.shape[0])

Rows before dropping: 16530
Rows after dropping: 15990


In [13]:
# Let's check we don't have any missing data
df_all_text_main['text'].isna().sum()

np.int64(0)

#### **TEXT CLEANING**

In [14]:
# Rename text column as orig_text
df_all_text_main.rename(columns={'text': 'orig_text'}, inplace=True)

In [15]:
# Map occurrences of the acronyms to their full forms in the text
def replace_acronyms(text, glossary=GLOSSARY_DICT):
	"""
	Replace acronyms in the text with their full forms based on the provided glossary.
	"""
	for acronym, full_form in glossary.items():
		text = re.sub(r'\b' + re.escape(acronym) + r'\b', full_form, text)
		# pattern = r'\b' + re.escape(acronym) + r'(s)?\b'
		# text = re.sub(pattern, full_form, text, flags=re.IGNORECASE)

	return text

In [16]:
def clean_text_func(text_list_raw):
    """
    Cleans a list of raw text by converting to lowercase and
    and filtering out stop words.

    Args:
        text_list_raw: List of raw text strings.

    Returns:
        text_list_clean: List of cleaned text strings.
    """
    stop_words = set(stopwords.words('english'))
    text_list_clean = []

    for text in text_list_raw:
        if text and text.lower() != "nan":
            text = replace_acronyms(text)
            text = text.lower()
            word_tokens = word_tokenize(text)
            filtered_tokens = [w for w in word_tokens if w not in stop_words]
            cleaned_text = " ".join(filtered_tokens)
            text_list_clean.append(cleaned_text)

    return text_list_clean

In [17]:
def clean_text_func2(text_list_raw):
    """
    Cleans a list of raw text by converting to lowercase and
    and filtering out stop words.

    Args:
        text_list_raw: List of raw text strings.

    Returns:
        text_list_clean: List of cleaned text strings.
    """
    text_list_clean = []

    for text in text_list_raw:
        if text and text.lower() != "nan":
            cleaned_text = replace_acronyms(text)
            cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
            text_list_clean.append(cleaned_text)

    return text_list_clean

In [18]:
# Conduct some basic text cleaning
clean_text = clean_text_func2(df_all_text_main['orig_text'].tolist())
df_all_text_main.insert(1, 'text', clean_text)  # Insert cleaned text at index 1
df_all_text_main

,orig_text,text,fiscal_period_ref,source,role,page,section,reporting_period,publication_date,bank,document_type,reference,source_type,is_comparative
0,"Thank you, Jenn, hello to everyone joining us ...","Thank you, Jenn, hello to everyone joining us ...",quarter,Jane Fraser,CEO,1.0,Prepared remarks,2023Q1,2023-04-14,Citigroup,transcript,"Jane Fraser (CEO)\nCitigroup, Q1, 2023 Earning...",internal,False
1,"First, our banking system as a whole is very s...","First, our banking system as a whole is very s...",quarter,Jane Fraser,CEO,1.0,Prepared remarks,2023Q1,2023-04-14,Citigroup,transcript,"Jane Fraser (CEO)\nCitigroup, Q1, 2023 Earning...",internal,False
2,The U.S. system comprises a healthy mix of com...,The U.S. system comprises a healthy mix of com...,quarter,Jane Fraser,CEO,1.0,Prepared remarks,2023Q1,2023-04-14,Citigroup,transcript,"Jane Fraser (CEO)\nCitigroup, Q1, 2023 Earning...",internal,False
3,I am pleased that Citi has been a source of st...,I am pleased that Citi has been a source of st...,quarter,Jane Fraser,CEO,1.0,Prepared remarks,2023Q1,2023-04-14,Citigroup,transcript,"Jane Fraser (CEO)\nCitigroup, Q1, 2023 Earning...",internal,False
4,We are in a position to play this role because...,We are in a position to play this role because...,quarter,Jane Fraser,CEO,1.0,Prepared remarks,2023Q1,2023-04-14,Citigroup,transcript,"Jane Fraser (CEO)\nCitigroup, Q1, 2023 Earning...",internal,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15985,Earnings gap between trading and investment ba...,Earnings gap between trading and investment ba...,,Joshua Franklin,,,,2025Q1,2025-04-08,JPMorgan,news,News article by Joshua Franklin\nTitle: Wall S...,external,True
15986,Profits at large US banks set to rise year on ...,Profits at large US banks set to rise year on ...,,Joshua Franklin,,,,2025Q1,2025-04-08,Citigroup,news,News article by Joshua Franklin\nTitle: Wall S...,external,True
15987,Profits at large US banks set to rise year on ...,Profits at large US banks set to rise year on ...,,Joshua Franklin,,,,2025Q1,2025-04-08,Bank of America,news,News article by Joshua Franklin\nTitle: Wall S...,external,True
15988,Profits at large US banks set to rise year on ...,Profits at large US banks set to rise year on ...,,Joshua Franklin,,,,2025Q1,2025-04-08,JPMorgan,news,News article by Joshua Franklin\nTitle: Wall S...,external,True


#### **SENTIMENT ANALYSIS**

In [19]:
def run_sentiment_analysis_on_gpu(text_subset, model_name, device_id):
	"""
	Run sentiment analysis on a subset of text using a specified model and device.
	"""
	tokenizer = AutoTokenizer.from_pretrained(model_name)
	model = AutoModelForSequenceClassification.from_pretrained(model_name)

	clf = pipeline("text-classification", model=model, tokenizer=tokenizer, device=device_id)

	results = []
	batch_size = 32

	for i in range(0, len(text_subset), batch_size):
		batch = text_subset[i:i+batch_size]
		batch_results = clf(batch, truncation=True, max_length=512)
		results.extend(batch_results)
		print(f"Device {device_id}: Processed {i + batch_size} reviews out of {len(text_subset)}")

	return results

In [20]:
# Create a Pydantic model for the topic label
class SentimentLabel(BaseModel):
	sentiment_label: Literal['Positive', 'Neutral', 'Negative']
	sentiment_rationale: str

In [21]:
def _make_sentiment_labelling_prompt(chunk, bank):
    """Define a prompt for the Sentiment labelling task."""
    return (
        "You are a financial language model tasked with determining sentiment toward a specific bank mentioned in a financial news excerpt.\n\n"
        "Here is the text excerpt you need to analyze:\n"
        f"Text: {chunk}\n\n"
        "Here is the target bank you need to focus on:\n"
        f"Target Bank: {bank}\n\n"
        "Instructions:\n"
        f"- Focus only on the sentiment **directed at {bank}**, not other entities.\n"
        "- Consider context such as performance, outlook, or comparisons.\n"
        f"- Avoid being influenced by other banks' outcomes unless directly relevant to {bank}.\n"
        "- Provide a clear sentiment label.\n"
        "- Include a brief rationale explaining your choice.\n"
    )

In [22]:
def label_sentiment_with_llm(chunk, bank, llm_backend, llm_model_name):
	"""Label a topic using the LLM."""
	prompt = _make_sentiment_labelling_prompt(chunk, bank)
	sentiment_llm = SentimentAnalysisLLM(
		sentiment_analysis_prompt=prompt,
		response_schema=SentimentLabel,
		backend=llm_backend,
		model_name=llm_model_name,
	)
	output = sentiment_llm.invoke()
	label = output.sentiment_label
	rationale = output.sentiment_rationale
	print(f"Sentiment with respect to {bank}: {chunk.strip()} --> Label: {label}")
	print(f"Rationale: {rationale}")
	return label, rationale

In [23]:
if RERUN_SENTIMENT_ANALYSIS:
    # First split the DataFrame between internal (transcripts) and external (news articles) text. Sentiment analysis of external text is more complicated as it contains comparative statements so we will use an LLM for that later.
    df_all_text_internal = df_all_text_main[df_all_text_main['source_type'] == 'internal'].copy()
    df_all_text_external = df_all_text_main[df_all_text_main['source_type'] == 'external'].copy()
    # We'll use finbert-tone for sentiment analysis which is the finetuned version of BERT for financial sentiment analysis
    model_name = "yiyanghkust/finbert-tone"

    # Check max token length for the model
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    print("Max token length of model:", model.config.max_position_embeddings)

    # Find max token length in the dataset
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    max_tokens = df_all_text_internal["text"].apply(
        lambda x: len(tokenizer(x, truncation=False)["input_ids"])
    ).max()
    print("Estimated max tokens in dataset:", max_tokens)

    # Split the reviews into two parts for parallel processing (2x GPUs)
    text_list = df_all_text_internal['text'].tolist()
    # Split list into 2 roughly equal parts
    midpoint = len(text_list) // 2
    text_list_split1 = text_list[:midpoint]
    text_list_split2 = text_list[midpoint:]

    # Run sentiment analysis on both subsets in parallel using ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=2) as executor:
        future_0 = executor.submit(run_sentiment_analysis_on_gpu, text_list_split1, model_name, 0)
        future_1 = executor.submit(run_sentiment_analysis_on_gpu, text_list_split2, model_name, 1)

        results_0 = future_0.result()
        results_1 = future_1.result()
        results = results_0 + results_1

    # Postprocess results and add to DataFrame
    results_clean = [d['label'] for d in results]
    df_all_text_internal['sentiment'] = results_clean

    # Now let's turn to the external text which is more complicated as it contains comparative statements. We will use an LLM to determine sentiment with respect to a specific bank mentioned in the text.

    # Initialize a dataframe to hold sentiment labels for external text
    sentiment_label_srs = pd.Series(index=df_all_text_external.index)
    rationale_srs = pd.Series(index=df_all_text_external.index)

    # Create a ThreadPoolExecutor to process docs concurrently
    with ThreadPoolExecutor(max_workers=20) as executor:
        futures = {}
        for row_idx, chunk, bank in df_all_text_external[['text','bank']].itertuples():
            future = executor.submit(
                label_sentiment_with_llm,
                chunk=chunk,
                bank=bank,
                llm_backend="gemini",
                llm_model_name="gemini-2.5-flash-preview-05-20"
            )
            futures[future] = row_idx

        # Wait for all futures to complete
        for future in as_completed(futures):
            try:
                row_idx = futures[future]
                label, rationale = future.result()  # This will raise an exception if the processing failed
                sentiment_label_srs.loc[row_idx] = label
                rationale_srs.loc[row_idx] = rationale
            except Exception as e:
                print(f"Error processing {futures[future]}: {e}")

    # Get index of section column
    df_all_text_external['sentiment'] = sentiment_label_srs
    df_all_text_external['sentiment_rationale'] = rationale_srs

    # Combine the internal and external sentiment results
    df_all_text_main = pd.concat([df_all_text_internal, df_all_text_external]) # Overwrite the main DataFrame with the combined results
    df_all_text_main.sort_index(inplace=True)

    # Save the results to a csv file
    df_all_text_main.to_csv("sentiment_analysis_results.csv", index=False)

else:
    # Load the results from the csv file
    df_all_text_main = pd.read_csv("sentiment_analysis_results.csv")

#### **TOPIC MODELLING - UNSUPERVISED**

In [24]:
if RERUN_UNSUPERVISED_TOPIC_MODELLING:
	# We'll use Fin-MPNET-Base for the embedding model in BERTopic as it has been tuned on financial data
	embedding_model_name = "mukaj/fin-mpnet-base"
	embedding_model = SentenceTransformer(embedding_model_name)
	# BERTopic uses UMAP, a dimensionality reduction technique, to reduce the dimensionality of the embeddings before clustering
	# UMAP introduces stochastic behaviour so we'll set a random seed for reproducibility
	# https://maartengr.github.io/BERTopic/getting_started/best_practices/best_practices.html#preventing-stochastic-behavior
	umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=0) # These are the default parameters for UMAP used in _bertopic.py with the additional parameter of random_state = 0

	# Run BERTopic to extract topics from the cleaned text
	model = BERTopic(verbose=True, embedding_model=embedding_model, umap_model=umap_model)
	model.fit(clean_text)
	topics, probabilities = model.transform(clean_text)

	df_topics = model.get_topic_freq()
	print("Top 10 topics and outliers: ", df_topics.head(11))
	pct_outliers = df_topics.loc[df_topics['Topic'] == -1, 'Count'].iloc[0]/df_topics['Count'].sum() * 100
	print(f"Percentage of outliers: {pct_outliers:.2f}%")

	df_all_text_main['topic_idx'] = topics

	print("---------------------TOPIC 0 ---------------------")
	print(model.get_topic(0))
	# Let's take a closer look a this topic
	for text in df_all_text_main.loc[df_all_text_main['topic_idx'] == 0, "text"].tolist():
		print("-----------------------------------------------------")
		print(text)

	print("---------------------TOPIC 1 ---------------------")
	print(model.get_topic(1))
	# Let's take a closer look a this topic
	for text in df_all_text_main.loc[df_all_text_main['topic_idx'] == 1, "text"].tolist():
		print("-----------------------------------------------------")
		print(text)

	# We can use a Hugging Face text generation model along with BERTopic's representation_model wrapper and pass this into BERTopic to summarise the topics for us
	# Since we'll be using Microsoft Phi 4 later, we'll use it here too.
	generator = pipeline(
		"text-generation",
		model="microsoft/Phi-4-mini-instruct",
		model_kwargs={"torch_dtype": "auto"},
		device_map="auto",
	)
	# Create prompt template to pass into BERTopic TextGeneration wrapper
	prompt_template = """<|system|>You are a helpful assistant who can succinctly describe the main topic covered by a set of customer reviews provided to you<|end|><|user|>I have a topic that contains the following documents: \n[DOCUMENTS]
	The topic is described by the following keywords: [KEYWORDS] Based on the above information, can you give a short label of the topic?<|end|><|assistant|>"""

	# Let's create the BERTopic TextGeneration wrapper now
	representation_model = TextGeneration(
		model=generator,
		prompt=prompt_template,
		pipeline_kwargs={"max_new_tokens": 200},
		random_state=0,
	)

	# Now let's rerun the BERTopic model exactly as before except this time we'll include the Phi 4 representation model above which will provide clean, interpretable topic labels for us
	umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=0) # These are the default parameters for UMAP used in _bertopic.py with the additional paramater of random_state = 0
	model = BERTopic(verbose=True, embedding_model=embedding_model, umap_model=umap_model, representation_model=representation_model)
	model.fit(clean_text)
	topics, probabilities = model.transform(clean_text)

	# Let's print out the topic summaries generated by Phi 4.
	topic_summaries_dict = defaultdict(list)
	max_topic_idx = df_all_text_main['topic_idx'].max()
	for i in range(max_topic_idx + 1):
		topic_summaries_dict["topic_idx"].append(i)
		topic_summaries_dict["reviews"].append(model.get_topic_freq(i))
		topic_summaries_dict["summary"].append(model.topic_representations_[i][0][0])
		print("-----------------------------------------------------")
		print(f"Topic {i}:")
		print(f"Number of reviews: {model.get_topic_freq(i)}")
		print(f"Summary: {model.topic_representations_[i][0][0]}")

	df_topic_summaries = pd.DataFrame(topic_summaries_dict)


In [25]:
if RERUN_UNSUPERVISED_TOPIC_MODELLING:
	# Plot the topics
	model.visualize_topics()


In [26]:
if RERUN_UNSUPERVISED_TOPIC_MODELLING:
	model.visualize_barchart()

In [27]:
if RERUN_UNSUPERVISED_TOPIC_MODELLING:
	model.visualize_heatmap(top_n_topics=10)

#### **TOPIC MODELLING - SEMI-SUPERVISED - GUIDED**

In [28]:
# Specify some seed topics
seed_topic_list = [
    # 0. Capital Adequacy
    ["capital", "tier 1", "tier 2", "risk-weighted assets", "capital buffer", "regulatory capital", "cet1", "capital ratio", "tangible book value", "leverage ratio", "supplementary leverage ratio", "leverage exposure", "capital constraints", "basel iii"],

    # 1. Liquidity Risk
    ["liquidity", "cash", "short-term funding", "liquid assets", "deposit outflows", "liquidity coverage ratio", "cash reserves"],

    # 2. Profitability
    ["profit", "earnings", "revenue", "margin", "return on equity", "net income", "operating income", "eps", "return on tangible common equity", "efficiency ratio"],

    # 3. Asset Quality / Credit Risk (NPLs)
    ["non-performing loans", "credit risk", "default", "delinquencies", "loan loss provisions", "impairment", "charge-offs", "allowance for credit losses", "write-downs", "reserve to funded loans"],

    # 4. Macroeconomic Risk / Interest Rates
    ["interest rates", "rate hikes", "monetary policy", "inflation", "yield curve", "economic outlook", "central bank", "tariffs"],

    # 5. Market Risk / Volatility
    ["market volatility", "value at risk", "trading losses", "asset prices", "derivatives", "hedging", "market downturn"],

    # 6. Operational Risk / Technology
    ["cybersecurity", "system failure", "fraud", "data breach", "internal control", "technology risk", "disruption"],

    # 7. Regulatory & Compliance
    ["regulatory", "compliance", "supervisory", "basel", "reporting standards", "audit", "oversight"],

    # 8. ESG / Reputation Risk
    ["sustainability", "climate risk", "reputation", "governance", "social responsibility", "stakeholders", "diversity"],

    # 9. Capital Returns / Shareholder Value
    ["dividends", "share buybacks", "capital return", "payout ratio", "shareholder value", "stock repurchase"],

    # 10. Strategic Risk / Business Model
    ["business strategy", "growth plans", "restructuring", "core business", "competitive advantage", "market positioning"]
]

In [29]:
if RERUN_GUIDED_TOPIC_MODELLING:
    # We'll use Fin-MPNET-Base for the embedding model in BERTopic as it has been tuned on financial data
    embedding_model_name = "mukaj/fin-mpnet-base"
    embedding_model = SentenceTransformer(embedding_model_name)

    # UMAP introduces stochastic behaviour so we'll set a random seed for reproducibility
    umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=0) # These are the default parameters for UMAP used in _bertopic.py with the additional parameter of random_state = 0
	# Run BERTopic to extract topics from the cleaned text
    model = BERTopic(
        verbose=True,
        nr_topics=50,
        seed_topic_list=seed_topic_list,
        embedding_model=embedding_model,
        umap_model=umap_model
        )
    model.fit(clean_text)
    topics, probabilities = model.transform(clean_text)

    df_topics = model.get_topic_freq()
    print(df_topics.head(50))

    pct_outliers = df_topics.loc[df_topics['Topic'] == -1, 'Count'].iloc[0]/df_topics['Count'].sum() * 100
    print(f"Percentage of outliers: {pct_outliers:.2f}%")

    df_all_text_main['topic_idx'] = topics

    print("---------------------TOPIC 0 ---------------------")
    print(model.get_topic(0))
    # Let's take a closer look a this topic
    for text in df_all_text_main.loc[df_all_text_main['topic_idx'] == 0, "text"].tolist():
        print("-----------------------------------------------------")
        print(text)

    print("---------------------TOPIC 1 ---------------------")
    print(model.get_topic(1))
    # Let's take a closer look a this topic
    for text in df_all_text_main.loc[df_all_text_main['topic_idx'] == 1, "text"].tolist():
        print("-----------------------------------------------------")
        print(text)

    # We can use a Hugging Face text generation model along with BERTopic's representation_model wrapper and pass this into BERTopic to summarise the topics for us
    # Since we'll be using Microsoft Phi 4 later, we'll use it here too.
    generator = pipeline(
        "text-generation",
        model="microsoft/Phi-4-mini-instruct",
        model_kwargs={"torch_dtype": "auto"},
        device_map="auto",
    )

    # Create prompt template to pass into BERTopic TextGeneration wrapper
    prompt_template = """<|system|>You are a helpful assistant who can succinctly describe the main topic covered by a set of customer reviews provided to you<|end|><|user|>I have a topic that contains the following documents: \n[DOCUMENTS]
    The topic is described by the following keywords: [KEYWORDS] Based on the above information, can you give a short label of the topic?<|end|><|assistant|>"""
    result = generator(prompt_template, max_new_tokens=200)
    result[0]["generated_text"]

    # Let's create the BERTopic TextGeneration wrapper now
    representation_model = TextGeneration(
        model=generator,
        prompt=prompt_template,
        pipeline_kwargs={"max_new_tokens": 200},
        random_state=0,
    )

    # Now let's rerun the BERTopic model exactly as before except this time we'll include the Phi 4 representation model above which will provide clean, interpretable topic labels for us

    umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=0) # These are the default parameters for UMAP used in _bertopic.py with the additional paramater of random_state = 0
    model = BERTopic(
        verbose=True,
        nr_topics= 50,
        seed_topic_list=seed_topic_list,
        embedding_model=embedding_model,
        umap_model=umap_model,
        representation_model=representation_model
    )
    model.fit(clean_text)
    topics, probabilities = model.transform(clean_text)

    # Let's print out the topic summaries generated by Phi 4.
    topic_summaries_dict = defaultdict(list)
    max_topic_idx = df_all_text_main['topic_idx'].max()
    for i in range(max_topic_idx + 1):
        topic_summaries_dict["topic_idx"].append(i)
        topic_summaries_dict["reviews"].append(model.get_topic_freq(i))
        topic_summaries_dict["summary"].append(model.topic_representations_[i][0][0])
        print("-----------------------------------------------------")
        print(f"Topic {i}:")
        print(f"Number of reviews: {model.get_topic_freq(i)}")
        print(f"Summary: {model.topic_representations_[i][0][0]}")

    df_topic_summaries = pd.DataFrame(topic_summaries_dict)




In [30]:
if RERUN_GUIDED_TOPIC_MODELLING:
	# Plot the topics
	model.visualize_topics()

In [31]:
if RERUN_GUIDED_TOPIC_MODELLING:
	model.visualize_barchart()

In [32]:
if RERUN_GUIDED_TOPIC_MODELLING:
	model.visualize_heatmap(top_n_topics=10)

#### **TOPIC MODELLING - SEMI-SUPERVISED - ZERO-SHOT**

In [33]:
topic_mapping_dict = {
    "Capital Adequacy": [
        "Capital", "Tier 1 Capital", "Tier 2 Capital", "Risk-Weighted Assets", "Capital Buffer",
        "Regulatory Capital", "CET1 Ratio", "Capital Ratio", "Tangible Book Value Per Share",
        "Leverage Ratio", "Supplementary Leverage Ratio", "Leverage Exposure", "Capital Constraints", "Basel III"
    ],

    "Liquidity Risk": [
        "Liquidity Risk", "Cash Position", "Short-Term Funding", "Liquid Assets", "Deposit Outflows",
        "Liquidity Coverage Ratio", "Cash Reserves"
    ],

    "Profitability": [
        "Profitability", "Earnings Performance", "Revenue Growth", "Profit Margin", "Net Interest Margin",
        "Return On Equity", "Net Income", "Operating Income", "Earnings Per Share",
        "Return On Tangible Common Equity", "Efficiency Ratio"
    ],

    "Asset Quality and Credit Risk": [
        "Non-Performing Loans", "Credit Risk", "Loan Defaults", "Delinquencies", "Loan Loss Provisions",
        "Impairments", "Charge-Offs", "Allowance For Credit Losses", "Loan Write-Downs",
        "Reserve To Funded Loans", "Stage 3 Loans"
    ],

    "Macroeconomic Risk": [
        "Rate Hikes", "Monetary Policy", "Inflation", "Economic Slowdown", "Central Bank Policy",
        "Unemployment", "Tariffs", "Macroeconomic Outlook", "Geopolitical Risk"
    ],

    "Interest Rate Risk": [
        "Interest Rate Risk", "Banking Book Interest Rate Risk", "IRRBB", "Net Interest Margin Sensitivity",
        "Repricing Gap", "Duration Mismatch", "Yield Curve Exposure", "Interest Rate Sensitivity",
        "Rate Shock Scenarios", "Basis Risk"
    ],

    "Market and Volatility Risk": [
        "Market Volatility", "Value At Risk", "Trading Losses", "Asset Price Fluctuation",
        "Derivative Exposure", "Hedging Strategy", "Market Downturn"
    ],

    "Operational Risk": [
        "Cybersecurity Threat", "System Failure", "Fraud Risk", "Data Breach",
        "Internal Controls", "Technology Risk", "Operational Disruption"
    ],

    "Regulatory & Compliance Risk": [
        "Regulatory Requirements", "Compliance Risk", "Supervisory Review", "Basel Framework",
        "Reporting Standards", "Audit Finding", "Regulatory Oversight"
    ],

    "ESG and Reputation Risk": [
        "Sustainability Goals", "Climate Risk", "Reputation Risk", "Corporate Governance",
        "Social Responsibility", "Stakeholder Engagement", "Diversity And Inclusion", "Community Impact"
    ],

    "Strategic and Business Model Risk": [
        "Business Strategy", "Growth Plans", "Corporate Restructuring", "Core Business Focus",
        "Competitive Positioning", "Market Entry Strategy"
    ],

    "Legal Risk": [
        "Litigation Risk", "Lawsuit", "Legal Proceedings", "Class Action",
        "Settlement", "Regulatory Investigation", "Legal Exposure", "Contractual Dispute", "Fines And Penalties"
    ]
}

In [34]:
topic_mapping_df = pd.DataFrame()
for risk_category, keywords in topic_mapping_dict.items():
	for keyword in keywords:
		topic_mapping_df = pd.concat(
			[topic_mapping_df, pd.DataFrame({"risk_category": [risk_category], "topic_label": [keyword]})],
			ignore_index=True
		)
topic_mapping_df

,risk_category,topic_label
0,Capital Adequacy,Capital
1,Capital Adequacy,Tier 1 Capital
2,Capital Adequacy,Tier 2 Capital
3,Capital Adequacy,Risk-Weighted Assets
4,Capital Adequacy,Capital Buffer
...,...,...
101,Legal Risk,Settlement
102,Legal Risk,Regulatory Investigation
103,Legal Risk,Legal Exposure
104,Legal Risk,Contractual Dispute


In [35]:
# Create a Pydantic model for the topic label
class TopicLabel(BaseModel):
	# topic_label: Optional[str]
	topic_label: str


In [36]:
def _make_topic_labelling_prompt(docs):
        """Define a prompt for the topic labelling task."""
        return (
        "You are a financial risk analyst. Below are excerpts from bank earnings calls and financial news articles that have been grouped together by a topic modeling algorithm.\n\n"
        "Your task is to assign a concise and reusable topic label (2–4 words) that best captures the shared subject matter of the group.\n\n"
        "Guidelines:\n"
        "- Focus only on substantive financial themes.\n"
        "- Ignore procedural, structural, or referential text (e.g., slide numbers, agenda items, transitions).\n"
        "- Use neutral phrasing that could apply across time periods and banks.\n"
        "- Avoid interpretations, adjectives, or qualifiers (e.g., 'significant', 'concerning').\n"
        "- Do not include the names of specific banks (e.g., Citigroup, JPMorgan) or people.\n"
        "- The label should summarize the theme, not individual facts.\n"
        # "- If there is no meaningful financial or business related content e.g. call logistics, return null.\n\n"
        "Here are the documents in this topic group:\n"
        "\n---\n".join(docs) + "\n"
        )



In [37]:
def label_topic_with_llm(topic_idx, topic_docs, llm_backend, llm_model_name):
	"""Label a topic using the LLM."""
	prompt = _make_topic_labelling_prompt(topic_docs)
	topic_label_llm = TopicLabellingLLM(
		topic_labelling_prompt=prompt,
		response_schema=TopicLabel,
		backend=llm_backend,
		model_name=llm_model_name,
	)
	topic_output = topic_label_llm.invoke()
	topic = topic_output.topic_label
	print(f"Topic {topic_idx}: {topic}")
	return topic

In [38]:
# Merge topics which are in the same cluster together
def merge_topics(df):
	"""
	Merge topics based on the cluster and risk category.
	"""
	if df['cluster'].iloc[0] == -1:
		# If the cluster is -1, it means it's an outlier, so we don't make any updates
		return df[['topic_idx', 'topic_label']].rename(columns={
			'topic_idx': 'post_merge_topic_idx',
			'topic_label': 'post_merge_topic_label'
		}).set_index(df.index)
	# For other clusters, get the most common topic index and associated topic label
	most_common_topic_idx = df['topic_idx'].mode()[0]
	most_common_topic_label = df['topic_label'].mode()[0]
	return pd.DataFrame({
		'post_merge_topic_idx': [most_common_topic_idx]*len(df),
		'post_merge_topic_label': [most_common_topic_label]*len(df),
	}, index=df.index)

In [39]:
if RERUN_ZERO_SHOT_TOPIC_MODELLING:
	# We'll use Fin-MPNET-Base for the embedding model in BERTopic as it has been tuned on financial data
	embedding_model_name = "mukaj/fin-mpnet-base"
	embedding_model = SentenceTransformer(embedding_model_name)

	# Set the UMAP model parameters with random_state for reproducibility
	umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=0) # These are the default parameters for UMAP used in _bertopic.py with the additional parameter of random_state = 0

	# Now we'll use the topic mapping values in a zero shot topic modelling approach and later map back to the broader topic categories
	zeroshot_topic_list = topic_mapping_df['topic_label'].tolist()

	# Run BERTopic to extract topics from the cleaned text
	model = BERTopic(
		verbose=True,
		# nr_topics=200,
		zeroshot_topic_list=zeroshot_topic_list,
		zeroshot_min_similarity= 0.5,
		embedding_model=embedding_model,
		umap_model=umap_model
		)
	model.fit(clean_text)
	topics, probabilities = model.transform(clean_text)

	df_topics = model.get_topic_freq()

	# Identify topics with only a few reviews
	minor_topics_list = df_topics.loc[df_topics['Count'] <= 3,"Topic"].tolist()
	print(minor_topics_list)

	pct_outliers = df_topics.loc[df_topics['Topic'] == -1, 'Count'].iloc[0]/df_topics['Count'].sum() * 100
	print(f"Percentage of outliers: {pct_outliers:.2f}%")

	df_all_text_main['topic_idx'] = topics

	print("---------------------TOPIC 0 ---------------------")
	print(model.get_topic(0))
	# Let's take a closer look at this topic
	for text in df_all_text_main.loc[df_all_text_main['topic_idx'] == 0, "text"].tolist():
		print("-----------------------------------------------------")
		print(text)

	print("---------------------TOPIC 1 ---------------------")
	print(model.get_topic(1))
	# Let's take a closer look at this topic
	for text in df_all_text_main.loc[df_all_text_main['topic_idx'] == 1, "text"].tolist():
		print("-----------------------------------------------------")
		print(text)

	topic_summary_df = model.get_topic_info()
	print("Topic Summary DataFrame:")
	print(topic_summary_df)

	# Print number of
	#  topics
	print("Number of zero shot topics: ", len(zeroshot_topic_list))

	# Get unsupervised topics
	sel_bool = ((topic_summary_df['Topic'] >= 0) & (~topic_summary_df['Name'].isin(zeroshot_topic_list)))
	print("Unsupervised topics:")
	print(topic_summary_df.loc[sel_bool, ['Topic','Count','Name']])
	unsupervised_topics_list = topic_summary_df.loc[sel_bool, 'Topic'].tolist()

	df_all_text_main_unsupervised = df_all_text_main[df_all_text_main['topic_idx'].isin(unsupervised_topics_list)]

	# Initialize a dataframe to hold topic labels
	unsupervised_topics_df = pd.DataFrame(columns=["topic_idx", "topic_label", "risk_category"])

	# Create a ThreadPoolExecutor to process docs concurrently
	with ThreadPoolExecutor(max_workers=10) as executor:
		futures = {}
		for unsupervised_topic_idx in unsupervised_topics_list:
			# Get the documents for the current topic
			topic_docs = df_all_text_main_unsupervised.loc[df_all_text_main_unsupervised['topic_idx'] == unsupervised_topic_idx, 'text'].tolist()
			future = executor.submit(
				label_topic_with_llm,
				topic_idx=unsupervised_topic_idx,
				topic_docs=topic_docs,
				llm_backend="gemini",
				llm_model_name="gemini-2.5-flash-preview-05-20"
				# llm_backend="openai",
				# llm_model_name="gpt-4o"
			)
			futures[future] = unsupervised_topic_idx

		# Wait for all futures to complete
		for future in as_completed(futures):
			try:
				topic_idx = futures[future]
				topic = future.result()  # This will raise an exception if the processing failed
				# Append the result to the labels_df
				unsupervised_topics_df = pd.concat(
					[unsupervised_topics_df, pd.DataFrame({
						"topic_idx": [topic_idx],
						"topic_label": [topic],
					})],
					ignore_index=True
				)
			except Exception as e:
				print(f"Error processing {futures[future]}: {e}")

	# Sort the DataFrame by topic_idx
	unsupervised_topics_df = unsupervised_topics_df.sort_values(by='topic_idx').reset_index(drop=True)

	# Zeroshot topic idxs and names
	zeroshot_topics_df = topic_summary_df.loc[topic_summary_df['Name'].isin(zeroshot_topic_list), ['Topic', 'Name']]
	# Rename columns for clarity
	zeroshot_topics_df.rename(columns={'Topic': 'topic_idx', 'Name': 'topic_label'}, inplace=True)
	# Add risk category based on the topic mapping
	zeroshot_topics_df = zeroshot_topics_df.merge(topic_mapping_df, on='topic_label', how='left')

	# Combine the zeroshot and unsupervised topics into a single DataFrame
	all_topics_df = pd.concat([zeroshot_topics_df, unsupervised_topics_df], ignore_index=True)

	# Let's reduce the number of topics by merging similar topic labels through clustering
	# Aid the topic merging process by replacing acronyms with their full forms
	# Apply the acronym replacement to the topic column
	all_topics_df['topic_label'] = all_topics_df['topic_label'].fillna('')  # Fill NaN values with empty strings
	all_topics_df['topic_label'] = all_topics_df['topic_label'].apply(lambda x: replace_acronyms(x, GLOSSARY_DICT))

	# # Perform clustering to identify similar topic labels
	# embedding_model_name = "mukaj/fin-mpnet-base"
	# # embedding_model_name = "FinLang/finance-embeddings-investopedia"
	# embedding_model = SentenceTransformer(embedding_model_name)

	all_topic_labels = all_topics_df['topic_label'].tolist()
	all_topic_label_embeddings = embedding_model.encode(all_topic_labels, normalize_embeddings=True)

	distance_matrix = cosine_distances(all_topic_label_embeddings)

	dbscan = DBSCAN(eps=0.25, min_samples=2, metric='precomputed') # EPS of 0.25 ensures that only very similar topic labels are clustered together
	cluster_labels = dbscan.fit_predict(distance_matrix)

	# Merge topics which are in the same cluster together
	all_topics_df['cluster'] = cluster_labels
	# all_topics_df.groupby('cluster').size()
	merged_df = all_topics_df.groupby(['cluster'], group_keys=False, dropna=False).apply(merge_topics).reset_index(drop=True)
	merged_topics_df = pd.concat([all_topics_df.reset_index(drop=True), merged_df], axis=1)

	# Print number of unique topics before and after merging
	print("Number of unique topics before merging:", all_topics_df['topic_idx'].nunique())
	print("Number of unique topics after merging:", merged_topics_df['post_merge_topic_idx'].nunique())

	# Merge the main DataFrame with the merged topics DataFrame
	df_all_text_topics = df_all_text_main.merge(
		merged_topics_df[['topic_idx', 'topic_label', 'post_merge_topic_idx', 'post_merge_topic_label']],
		on='topic_idx',
		how='left'
	)

	# Drop rows where the topic label is NaN (these don't contain any substantive financial themes e.g. transition statements)


	# Save the DataFrame with topics to a CSV file
	df_all_text_topics.to_csv("zero_shot_topic_modelling_with_merging.csv", index=False)

else:
	# Load the DataFrame with topics from the CSV file
	df_all_text_topics = pd.read_csv("zero_shot_topic_modelling_with_merging.csv")







In [40]:
class RiskScore(BaseModel):
	risk_score: float = Field(..., ge=0, le=1)

class RiskCategoryScoring(BaseModel):
	capital_adequacy: RiskScore
	liquidity_risk: RiskScore
	profitability: RiskScore
	asset_quality_and_credit_risk: RiskScore
	macroeconomic_risk: RiskScore
	interest_rate_risk: RiskScore
	market_and_volatility_risk: RiskScore
	operational_risk: RiskScore
	regulatory_and_compliance_risk: RiskScore
	esg_and_reputation_risk: RiskScore
	strategic_and_business_model_risk: RiskScore
	scoring_rationale: str

class RiskCategoryLabel(BaseModel):
	risk_category: Optional[Literal['Capital Adequacy', 'Liquidity Risk', 'Profitability', 'Asset Quality and Credit Risk', 'Macroeconomic Risk', 'Interest Rate Risk', 'Market and Volatility Risk', 'Operational Risk', 'Regulatory & Compliance Risk', 'ESG and Reputation Risk', 'Strategic and Business Model Risk', 'Legal Risk']]
	# risk_category_rationale: str



In [41]:
def _make_risk_category_scoring_prompt(chunk):
	"""Define a prompt for the topic labelling task."""
	return (
        "You are a bank risk analyst with expertise in the Prudential Regulation Authority (PRA) rulebook.\n\n"
        "Below is an excerpt from a bank earnings call or financial news article:\n"
        f"{chunk}\n\n"
        "Your task is to assess how relevant this text is to each of the following risk categories defined by the PRA. For each, score relevance from 0 (not relevant) to 1 (highly relevant).\n\n"
        "Here are summary definitions of the risk categories:\n"
        "- Capital Adequacy: The risk that a bank holds insufficient capital to absorb losses while maintaining confidence and meeting regulatory requirements.\n"
        "- Liquidity Risk: The risk that a firm is unable to meet its financial obligations as they fall due, without incurring unacceptable losses.\n"
        "- Profitability: The risk that the firm’s earnings capacity is insufficient to support its strategic goals, capital generation, or risk absorption.\n"
        "- Asset Quality and Credit Risk: The risk of borrower default or asset impairment, affecting the quality of the firm’s loan book or other exposures.\n"
        "- Macroeconomic Risk: Risks arising from systemic economic factors such as inflation, GDP volatility, or geopolitical events.\n"
        "- Interest Rate Risk: The risk to earnings or capital from movements in interest rates, including re-pricing mismatches and basis risk.\n"
        "- Market and Volatility Risk: The risk of losses from movements in market prices, including volatility in equity, FX, or commodity markets.\n"
        "- Operational Risk: The risk of loss due to failed internal processes, people, systems, or external events, including cyber threats.\n"
        "- Regulatory & Compliance Risk: The risk of legal or regulatory sanctions due to non-compliance with laws, rules, or supervisory expectations.\n"
        "- ESG and Reputation Risk: The risk arising from environmental, social, governance, or reputational issues that could impact stakeholder trust or regulatory standing.\n"
        "- Strategic and Business Model Risk: The risk that the firm’s business model is not sustainable, misaligned with market dynamics, or exposed to poor strategic decisions.\n"
        "- Legal Risk: The risk of loss from litigation, contract disputes, or failure to meet legal obligations.\n\n"
        "Provide a single paragraph rationale explaining why the scores were assigned — referencing the most relevant parts of the text."
	)

def _make_risk_category_labelling_prompt(chunk):
	"""Define a prompt for the risk category labelling task."""
	return (
        "You are a bank risk analyst familiar with PRA regulations.\n\n"
        "Classify the following text into the **single most relevant** financial risk category.\n"
        "If none apply, return 'risk_category': null.\n\n"
        f"Text:\n{chunk}\n\n"
        "Categories:\n"
        "- Capital Adequacy: Insufficient capital to absorb losses.\n"
        "- Liquidity Risk: Inability to meet obligations on time.\n"
        "- Profitability: Earnings too weak to support strategy or risk.\n"
        "- Asset Quality and Credit Risk: Defaults or asset impairments.\n"
        "- Macroeconomic Risk: Impact from broad economic conditions.\n"
        "- Interest Rate Risk: Losses due to rate changes.\n"
        "- Market and Volatility Risk: Losses from price movements.\n"
        "- Operational Risk: Failures in processes, people, or systems.\n"
        "- Regulatory & Compliance Risk: Breach of laws or rules.\n"
        "- ESG and Reputation Risk: Environmental, social, or reputational damage.\n"
        "- Strategic and Business Model Risk: Unsustainable or misaligned strategy.\n"
        "- Legal Risk: Litigation or contractual disputes.\n"
		# "You are a bank risk analyst with expertise in the Prudential Regulation Authority (PRA) rulebook.\n\n"
        # "Below is an excerpt from a bank earnings call or financial news article:\n"
        # f"{chunk}\n\n"
        # "Your task is to classify the topic under the **single most relevant** financial risk category.\n\n"
		# "Choose one from the following categories:\n"
		# "- Capital Adequacy: The risk that a bank holds insufficient capital to absorb losses while maintaining confidence and meeting regulatory requirements.\n"
        # "- Liquidity Risk: The risk that a firm is unable to meet its financial obligations as they fall due, without incurring unacceptable losses.\n"
        # "- Profitability: The risk that the firm’s earnings capacity is insufficient to support its strategic goals, capital generation, or risk absorption.\n"
        # "- Asset Quality and Credit Risk: The risk of borrower default or asset impairment, affecting the quality of the firm’s loan book or other exposures.\n"
        # "- Macroeconomic Risk: Risks arising from systemic economic factors such as inflation, GDP volatility, or geopolitical events.\n"
        # "- Interest Rate Risk: The risk to earnings or capital from movements in interest rates, including re-pricing mismatches and basis risk.\n"
        # "- Market and Volatility Risk: The risk of losses from movements in market prices, including volatility in equity, FX, or commodity markets.\n"
        # "- Operational Risk: The risk of loss due to failed internal processes, people, systems, or external events, including cyber threats.\n"
        # "- Regulatory & Compliance Risk: The risk of legal or regulatory sanctions due to non-compliance with laws, rules, or supervisory expectations.\n"
        # "- ESG and Reputation Risk: The risk arising from environmental, social, governance, or reputational issues that could impact stakeholder trust or regulatory standing.\n"
        # "- Strategic and Business Model Risk: The risk that the firm’s business model is not sustainable, misaligned with market dynamics, or exposed to poor strategic decisions.\n"
        # "- Legal Risk: The risk of loss from litigation, contract disputes, or failure to meet legal obligations.\n\n"
		# "If the text does not fit under any risk category (e.g., greetings, procedural comments), set the 'risk_category' to null.\n"
		# # "Also provide a brief rationale explaining your choice."

	)

In [42]:
def score_risk_categories_with_llm(chunk, llm_backend, llm_model_name):
	"""Label a risk category using the LLM."""
	prompt = _make_risk_category_scoring_prompt(chunk)
	risk_label_llm = RiskLabellingLLM(
		risk_labelling_prompt=prompt,
		response_schema=RiskCategoryScoring,
		backend=llm_backend,
		model_name=llm_model_name,
	)
	risk_output = risk_label_llm.invoke()
	risk_scores_dict = dict(risk_output)
	# Reformat the risk scores dictionary to only include the risk scores while keeping rationale as is
	risk_scores_dict = {k: v.risk_score if isinstance(v, RiskScore) else v for k, v in risk_scores_dict.items()}
	return risk_scores_dict

def label_risk_category_with_llm(chunk, llm_backend, llm_model_name):
	"""Label a risk category using the LLM."""
	prompt = _make_risk_category_labelling_prompt(chunk)
	risk_label_llm = RiskLabellingLLM(
		risk_labelling_prompt=prompt,
		response_schema=RiskCategoryLabel,
		backend=llm_backend,
		model_name=llm_model_name,
	)
	risk_output = risk_label_llm.invoke()
	risk_category_dict = dict(risk_output)
	# time.sleep(10)  # Sleep for a few seconds to avoid hitting rate limits

	return risk_category_dict

In [43]:
if RERUN_RISK_CATEGORY_LABELLING:
	if RUN_RISK_CATEGORY_SCORING:
		print("RUNNING RISK CATEGORY SCORING")
		# Now let's put each text chunk into a broader risk category
		# Initialize a dataframe to hold risk category labels
		cols = list(RiskCategoryScoring.model_fields.keys())
		risk_category_df = pd.DataFrame(index=df_all_text_topics.index, columns=cols)

		# Get text chunks from the DataFrame
		text_chunks_list = df_all_text_topics['text'].tolist()
		text_chunks_list = text_chunks_list[:100]  # Limit to first 100 chunks for testing; remove this line for full run

		# Create a ThreadPoolExecutor to process docs concurrently
		with ThreadPoolExecutor(max_workers=25) as executor:
			futures = {}
			for idx, text_chunk in enumerate(text_chunks_list):
				future = executor.submit(
					score_risk_categories_with_llm,
					chunk=text_chunk,
					llm_backend="gemini",
					# llm_model_name="gemini-2.5-flash-preview-05-20"
					llm_model_name="gemini-2.5-pro-preview-06-05"
				)
				futures[future] = idx

			# Wait for all futures to complete
			for future in as_completed(futures):
				try:
					idx = futures[future]
					risk_scores_dict = future.result()  # This will raise an exception if the processing failed
					# Append the result to the labels_df
					risk_category_df.loc[idx] = risk_scores_dict
				except Exception as e:
					print(f"Error processing {futures[future]}: {e}")

		# Add the risk category labels to the main DataFrame
		df_all_text_topics = df_all_text_topics.join(risk_category_df, how='left')


	else:
		print("RUNNING SIMPLE RISK CATEGORY LABELLING")
		# Now let's put each text chunk into a broader risk category
		# Initialize a dataframe to hold risk category labels
		cols = list(RiskCategoryLabel.model_fields.keys())
		risk_category_df = pd.DataFrame(index=df_all_text_topics.index, columns=cols)

		# Get text chunks from the DataFrame
		text_chunks_list = df_all_text_topics['text'].tolist()
		# text_chunks_list = text_chunks_list[:400]  # Limit to first 200 chunks for testing; remove this line for full run

		# Create a ThreadPoolExecutor to process docs concurrently
		with ThreadPoolExecutor(max_workers=25) as executor:
			futures = {}
			for idx, text_chunk in enumerate(text_chunks_list):
				future = executor.submit(
					label_risk_category_with_llm,
					chunk=text_chunk,
					# llm_backend="gemini",
					# llm_model_name="gemini-2.5-flash-preview-05-20"
					# llm_model_name="gemini-2.5-pro-preview-06-05"
					llm_backend="openai",
					# llm_model_name="gpt-4o"
					llm_model_name="gpt-4.1-mini"
				)
				futures[future] = idx

			# Wait for all futures to complete
			count = 0
			for future in as_completed(futures):
				try:
					idx = futures[future]
					risk_category_dict = future.result()  # This will raise an exception if the processing failed
					# Append the result to the labels_df
					risk_category_df.loc[idx] = risk_category_dict
					count += 1
					print(f"Processed {count}/{len(text_chunks_list)}")
				except Exception as e:
					print(f"Error processing {futures[future]}: {e}")

		# Add the risk category labels to the main DataFrame
		df_all_text_topics = df_all_text_topics.join(risk_category_df, how='left')

	# Save the DataFrame with risk category labels to a CSV file
	df_all_text_topics.to_csv("zero_shot_topic_modelling_with_merging_risk_categories.csv", index=False)
else:
	# Load the risk category labels from the CSV file
	df_all_text_topics = pd.read_csv("zero_shot_topic_modelling_with_merging_risk_categories.csv")

In [44]:
df_all_text_topics

,orig_text,text,fiscal_period_ref,source,role,page,section,reporting_period,publication_date,bank,...,reference,source_type,is_comparative,sentiment,sentiment_rationale,topic_idx,topic_label,post_merge_topic_idx,post_merge_topic_label,risk_category
0,"Thank you, Jenn, hello to everyone joining us ...","Thank you, Jenn, hello to everyone joining us ...",quarter,Jane Fraser,CEO,1.0,Prepared remarks,2023Q1,14/04/2023,Citigroup,...,"Jane Fraser (CEO)\nCitigroup, Q1, 2023 Earning...",internal,False,Positive,NaN,39,Macroeconomic Outlook,39.0,Macroeconomic Outlook,Macroeconomic Risk
1,"First, our banking system as a whole is very s...","First, our banking system as a whole is very s...",quarter,Jane Fraser,CEO,1.0,Prepared remarks,2023Q1,14/04/2023,Citigroup,...,"Jane Fraser (CEO)\nCitigroup, Q1, 2023 Earning...",internal,False,Positive,NaN,208,U.S. Retail Banking,208.0,U.S. Retail Banking,Macroeconomic Risk
2,The U.S. system comprises a healthy mix of com...,The U.S. system comprises a healthy mix of com...,quarter,Jane Fraser,CEO,1.0,Prepared remarks,2023Q1,14/04/2023,Citigroup,...,"Jane Fraser (CEO)\nCitigroup, Q1, 2023 Earning...",internal,False,Positive,NaN,208,U.S. Retail Banking,208.0,U.S. Retail Banking,Regulatory & Compliance Risk
3,I am pleased that Citi has been a source of st...,I am pleased that Citi has been a source of st...,quarter,Jane Fraser,CEO,1.0,Prepared remarks,2023Q1,14/04/2023,Citigroup,...,"Jane Fraser (CEO)\nCitigroup, Q1, 2023 Earning...",internal,False,Positive,NaN,275,Transformation Initiatives,275.0,Operational Transformation,ESG and Reputation Risk
4,We are in a position to play this role because...,We are in a position to play this role because...,quarter,Jane Fraser,CEO,1.0,Prepared remarks,2023Q1,14/04/2023,Citigroup,...,"Jane Fraser (CEO)\nCitigroup, Q1, 2023 Earning...",internal,False,Positive,NaN,333,Global Banking Strategy,333.0,Global Banking Strategy,Strategic and Business Model Risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15985,Earnings gap between trading and investment ba...,Earnings gap between trading and investment ba...,NaN,Joshua Franklin,NaN,NaN,NaN,2025Q1,08/04/2025,JPMorgan,...,News article by Joshua Franklin\nTitle: Wall S...,external,True,Neutral,The text discusses general trends in trading a...,260,Banking Revenue Performance,116.0,Bank Performance,Profitability
15986,Profits at large US banks set to rise year on ...,Profits at large US banks set to rise year on ...,NaN,Joshua Franklin,NaN,NaN,NaN,2025Q1,08/04/2025,Citigroup,...,News article by Joshua Franklin\nTitle: Wall S...,external,True,Negative,The text states that 'Citigroup's profits are ...,270,Capital Markets Performance,230.0,Capital Markets Performance,Profitability
15987,Profits at large US banks set to rise year on ...,Profits at large US banks set to rise year on ...,NaN,Joshua Franklin,NaN,NaN,NaN,2025Q1,08/04/2025,Bank of America,...,News article by Joshua Franklin\nTitle: Wall S...,external,True,Positive,The text states that 'Bank of America is the s...,270,Capital Markets Performance,230.0,Capital Markets Performance,Profitability
15988,Profits at large US banks set to rise year on ...,Profits at large US banks set to rise year on ...,NaN,Joshua Franklin,NaN,NaN,NaN,2025Q1,08/04/2025,JPMorgan,...,News article by Joshua Franklin\nTitle: Wall S...,external,True,Positive,JPMorgan is consistently highlighted as contri...,270,Capital Markets Performance,230.0,Capital Markets Performance,Profitability


#### **MULTI-TOPIC LABELLING**

In [45]:
if RERUN_MULTI_TOPIC_LABELLING:
	# TOPIC RELEVANCE
	embedding_model_name = "mukaj/fin-mpnet-base"
	embedding_model = SentenceTransformer(embedding_model_name)
	text_embeddings = embedding_model.encode(df_all_text_topics['text'].tolist())
	topic_labels = df_all_text_topics['post_merge_topic_label'].dropna().unique().tolist()
	topic_embeddings = embedding_model.encode(topic_labels)
	similarities_arr = cosine_similarity(text_embeddings, topic_embeddings)
	similarities_df = pd.DataFrame(similarities_arr, columns=topic_labels)
	threshold = 0.35
	# similarities_df_flags = (similarities_df > threshold)*1
	# df_all_text_topics_flags = df_all_text_topics.copy()
	# df_all_text_topics_flags = pd.concat([df_all_text_topics_flags, similarities_df_flags], axis=1)
	# df_all_text_topics_flags.to_csv("df_all_text_topics_flags.csv", index=False)

	similarities_df_relevance = similarities_df[(similarities_df > threshold)].fillna(0).copy()
	df_all_text_topics_relevance = df_all_text_topics.copy()
	df_all_text_topics_relevance = pd.concat([df_all_text_topics_relevance, similarities_df_relevance], axis=1)

	# Let's drop any text chunks which don't have a clear risk category
	df_all_text_topics_relevance = df_all_text_topics_relevance.dropna(subset=['risk_category'])
	df_all_text_topics_relevance.reset_index(drop=True, inplace=True)

	# Save the DataFrame with topic relevance to csv/parquet
	fname = "multi_topic_modelling_with_relevance"
	fpath_parquet = os.path.join(APP_DATA_FOLDER_PATH, fname + ".parquet")
	fpath_csv = os.path.join(APP_DATA_FOLDER_PATH, fname + ".csv")
	df_all_text_topics_relevance.to_parquet(fpath_parquet)
	df_all_text_topics_relevance.to_csv(fpath_csv, index=False)

	# Aggregate the topic relevance scores to the risk category level
	first_subtopic_col = df_all_text_topics_relevance.columns.get_loc('risk_category') + 1
	subtopic_cols = df_all_text_topics_relevance.columns[first_subtopic_col:].tolist()
	grpby_cols = ['bank', 'reporting_period', 'risk_category']
	df_all_text_topics_relevance_quarter_agg = df_all_text_topics_relevance.groupby(grpby_cols).size().to_frame("risk_count")
	df_all_text_topics_relevance_quarter_agg2 = df_all_text_topics_relevance.groupby(grpby_cols)[subtopic_cols].sum()
	df_all_text_topics_relevance_quarter_agg = df_all_text_topics_relevance_quarter_agg.join(df_all_text_topics_relevance_quarter_agg2).reset_index()

	# We'll also groupby whether the source is internal or external and append to the aggregated DataFrame
	df_all_text_topics_relevance_quarter_agg['source_type'] = "all"
	grpby_cols = ['bank', 'reporting_period', 'risk_category', 'source_type']
	df_all_text_topics_relevance_quarter_agg_source = df_all_text_topics_relevance.groupby(grpby_cols).size().to_frame("risk_count")
	df_all_text_topics_relevance_quarter_agg_source2 = df_all_text_topics_relevance.groupby(grpby_cols)[subtopic_cols].sum()
	df_all_text_topics_relevance_quarter_agg_source = df_all_text_topics_relevance_quarter_agg_source.join(
		df_all_text_topics_relevance_quarter_agg_source2).reset_index()
	# Append the source type aggregated DataFrame to the main aggregated DataFrame
	df_all_text_topics_relevance_quarter_agg = pd.concat(
		[df_all_text_topics_relevance_quarter_agg, df_all_text_topics_relevance_quarter_agg_source],
		ignore_index=True
	)

	# Save the aggregated DataFrame to a csv and parquet
	fname="multi_topic_modelling_with_relevance_quarter_agg"
	fpath_parquet = os.path.join(APP_DATA_FOLDER_PATH, fname + ".parquet")
	fpath_csv = os.path.join(APP_DATA_FOLDER_PATH, fname + ".csv")
	df_all_text_topics_relevance_quarter_agg.to_parquet(fpath_parquet)
	df_all_text_topics_relevance_quarter_agg.to_csv(fpath_csv, index=False)

	# NORMALIZED TOPIC RELEVANCE
	# Normalize the subtopic columns so that they add to 1 for each row
	df_all_text_topics_relevance_norm = df_all_text_topics_relevance.copy()
	subtopic_sum_srs = df_all_text_topics_relevance_norm[subtopic_cols].sum(axis=1)
	df_all_text_topics_relevance_norm[subtopic_cols] = df_all_text_topics_relevance_norm[subtopic_cols].div(subtopic_sum_srs, axis=0).fillna(0)  # Normalize the subtopic columns by dividing by the sum of each row, filling NaN with 0

	# Save the DataFrame with topic relevance to csv/parquet
	fname = "multi_topic_modelling_with_relevance_norm"
	fpath_parquet = os.path.join(APP_DATA_FOLDER_PATH, fname + ".parquet")
	fpath_csv = os.path.join(APP_DATA_FOLDER_PATH, fname + ".csv")
	df_all_text_topics_relevance_norm.to_parquet(fpath_parquet)
	df_all_text_topics_relevance_norm.to_csv(fpath_csv, index=False)

	# Aggregate the topic relevance scores to the risk category level
	grpby_cols = ['bank', 'reporting_period', 'risk_category']
	df_all_text_topics_relevance_quarter_agg_norm = df_all_text_topics_relevance.groupby(grpby_cols).size().to_frame("risk_count").reset_index() # NOTE: not using the normalized version of the relevance data here
	grpby_cols2 = ['bank', 'reporting_period']
	df_all_text_topics_relevance_quarter_agg_norm['risk_proportion'] = df_all_text_topics_relevance_quarter_agg_norm['risk_count']/df_all_text_topics_relevance_quarter_agg_norm.groupby(grpby_cols2)['risk_count'].transform('sum')
	df_all_text_topics_relevance_quarter_agg_norm.drop(columns=['risk_count'], inplace=True) # Drop the risk_count column as we don't need it anymore
	df_all_text_topics_relevance_quarter_agg_norm.set_index(grpby_cols, inplace=True)

	df_all_text_topics_relevance_quarter_agg_norm2 = df_all_text_topics_relevance.groupby(grpby_cols)[subtopic_cols].sum() # NOTE: not using the normalized version of the relevance data here
	df_all_text_topics_relevance_quarter_agg_norm2[subtopic_cols] = df_all_text_topics_relevance_quarter_agg_norm2[subtopic_cols].div(df_all_text_topics_relevance_quarter_agg_norm2[subtopic_cols].sum(axis=1), axis=0)
	df_all_text_topics_relevance_quarter_agg_norm = df_all_text_topics_relevance_quarter_agg_norm.join(df_all_text_topics_relevance_quarter_agg_norm2).reset_index()

	# We'll also groupby whether the source is internal or external and append to the aggregated DataFrame
	df_all_text_topics_relevance_quarter_agg_norm['source_type'] = "all"
	grpby_cols = ['bank', 'reporting_period', 'risk_category', 'source_type']
	df_all_text_topics_relevance_quarter_agg_norm_source = df_all_text_topics_relevance.groupby(grpby_cols).size().to_frame("risk_count").reset_index() # NOTE: not using the normalized version of the relevance data here
	grpby_cols2 = ['bank', 'reporting_period']
	df_all_text_topics_relevance_quarter_agg_norm_source['risk_proportion'] = df_all_text_topics_relevance_quarter_agg_norm_source['risk_count']/df_all_text_topics_relevance_quarter_agg_norm_source.groupby(grpby_cols2)['risk_count'].transform('sum')
	df_all_text_topics_relevance_quarter_agg_norm_source.drop(columns=['risk_count'], inplace=True) # Drop the risk_count column as we don't need it anymore
	df_all_text_topics_relevance_quarter_agg_norm_source.set_index(grpby_cols, inplace=True)
	df_all_text_topics_relevance_quarter_agg_norm_source2 = df_all_text_topics_relevance.groupby(grpby_cols)[subtopic_cols].sum() # NOTE: not using the normalized version of the relevance data here
	df_all_text_topics_relevance_quarter_agg_norm_source2[subtopic_cols] = df_all_text_topics_relevance_quarter_agg_norm_source2[subtopic_cols].div(df_all_text_topics_relevance_quarter_agg_norm_source2[subtopic_cols].sum(axis=1), axis=0)
	df_all_text_topics_relevance_quarter_agg_norm_source = df_all_text_topics_relevance_quarter_agg_norm_source.join(df_all_text_topics_relevance_quarter_agg_norm_source2).reset_index()
	# Append the source type aggregated DataFrame to the main aggregated DataFrame
	df_all_text_topics_relevance_quarter_agg_norm = pd.concat(
		[df_all_text_topics_relevance_quarter_agg_norm, df_all_text_topics_relevance_quarter_agg_norm_source],
		ignore_index=True
	)

	# Save the aggregated DataFrame to a csv and parquet
	fname="multi_topic_modelling_with_relevance_quarter_agg_norm"
	fpath_parquet = os.path.join(APP_DATA_FOLDER_PATH, fname + ".parquet")
	fpath_csv = os.path.join(APP_DATA_FOLDER_PATH, fname + ".csv")
	df_all_text_topics_relevance_quarter_agg_norm.to_parquet(fpath_parquet)
	df_all_text_topics_relevance_quarter_agg_norm.to_csv(fpath_csv, index=False)

	# NET SENTIMENT
	# Add sentiment weighting to the topic relevance scores
	df_all_text_topics_relevance_sentiment = df_all_text_topics_relevance.copy()
	# df_all_text_topics_relevance_sentiment = df_all_text_topics_relevance_normalized.copy()
	df_all_text_topics_relevance_sentiment['sentiment_score'] = df_all_text_topics_relevance_sentiment['sentiment'].replace(
		{'Negative': -1, 'Neutral': 0, 'Positive': 1}
	)
	df_all_text_topics_relevance_sentiment[subtopic_cols] = df_all_text_topics_relevance_sentiment[subtopic_cols].mul(
		df_all_text_topics_relevance_sentiment['sentiment_score'], axis=0
	)

	# Save the DataFrame with sentiment weighting to csv and parquet
	fname = "multi_topic_modelling_with_relevance_sentiment"
	fpath_parquet = os.path.join(APP_DATA_FOLDER_PATH, fname + ".parquet")
	fpath_csv = os.path.join(APP_DATA_FOLDER_PATH, fname + ".csv")
	df_all_text_topics_relevance_sentiment.to_parquet(fpath_parquet)
	df_all_text_topics_relevance_sentiment.to_csv(fpath_csv, index=False)
	# df_all_text_topics_relevance_sentiment.to_csv("multi_topic_modelling_with_relevance_normalized_sentiment.csv", index=False)

	# Aggregate the topic relevance scores to the risk category level
	grpby_cols = ['bank', 'reporting_period', 'risk_category']
	df_all_text_topics_relevance_sentiment_quarter_agg = df_all_text_topics_relevance_sentiment.groupby(grpby_cols)['sentiment_score'].sum().to_frame()
	df_all_text_topics_relevance_sentiment_quarter_agg2 = df_all_text_topics_relevance_sentiment.groupby(grpby_cols)[subtopic_cols].sum()
	df_all_text_topics_relevance_sentiment_quarter_agg = df_all_text_topics_relevance_sentiment_quarter_agg.join(df_all_text_topics_relevance_sentiment_quarter_agg2).reset_index()

	# We'll also groupby whether the source is internal or external and append to the aggregated DataFrame
	df_all_text_topics_relevance_sentiment_quarter_agg['source_type'] = "all"
	grpby_cols = ['bank', 'reporting_period', 'risk_category', 'source_type']
	df_all_text_topics_relevance_sentiment_quarter_agg_source = df_all_text_topics_relevance_sentiment.groupby(grpby_cols)['sentiment_score'].sum().to_frame()
	df_all_text_topics_relevance_sentiment_quarter_agg_source2 = df_all_text_topics_relevance_sentiment.groupby(grpby_cols)[subtopic_cols].sum()
	df_all_text_topics_relevance_sentiment_quarter_agg_source = df_all_text_topics_relevance_sentiment_quarter_agg_source.join(
		df_all_text_topics_relevance_sentiment_quarter_agg_source2).reset_index()
	# Append the source type aggregated DataFrame to the main aggregated DataFrame
	df_all_text_topics_relevance_sentiment_quarter_agg = pd.concat(
		[df_all_text_topics_relevance_sentiment_quarter_agg, df_all_text_topics_relevance_sentiment_quarter_agg_source],
		ignore_index=True
	)

	# Save the aggregated DataFrame to a csv and parquet
	fname="multi_topic_modelling_with_relevance_sentiment_quarter_agg"
	fpath_parquet = os.path.join(APP_DATA_FOLDER_PATH, fname + ".parquet")
	fpath_csv = os.path.join(APP_DATA_FOLDER_PATH, fname + ".csv")
	df_all_text_topics_relevance_sentiment_quarter_agg.to_parquet(fpath_parquet)
	df_all_text_topics_relevance_sentiment_quarter_agg.to_csv(fpath_csv, index=False)

	# NORMALIZED NET SENTIMENT

	# Normalize the subtopic columns so that they add to 1 for each row
	df_all_text_topics_relevance_sentiment_norm = df_all_text_topics_relevance_sentiment.copy()
	subtopic_sum_srs = df_all_text_topics_relevance_sentiment_norm[subtopic_cols].abs().sum(axis=1) # Taking the absolute value here to preserve sign of sentiment score
	df_all_text_topics_relevance_sentiment_norm[subtopic_cols] = df_all_text_topics_relevance_sentiment_norm[subtopic_cols].div(subtopic_sum_srs, axis=0).fillna(0)

	# Save the DataFrame with topic relevance to csv/parquet
	fname = "multi_topic_modelling_with_relevance_sentiment_norm"
	fpath_parquet = os.path.join(APP_DATA_FOLDER_PATH, fname + ".parquet")
	fpath_csv = os.path.join(APP_DATA_FOLDER_PATH, fname + ".csv")
	df_all_text_topics_relevance_sentiment_norm.to_parquet(fpath_parquet)
	df_all_text_topics_relevance_sentiment_norm.to_csv(fpath_csv, index=False)

	# Aggregate the normalized sentiment scores to the risk category level
	df_all_text_topics_relevance_sentiment_quarter_agg_norm = df_all_text_topics_relevance_sentiment.copy()
	grpby_cols = ['bank', 'reporting_period', 'risk_category']
	df_all_text_topics_relevance_sentiment_quarter_agg_norm['sentiment_score'] = df_all_text_topics_relevance_sentiment_quarter_agg_norm['sentiment_score'].div(
		df_all_text_topics_relevance_sentiment_quarter_agg_norm.groupby(grpby_cols)['sentiment_score'].transform('count'), axis=0 # Normalize the sentiment score by dividing by the number of observations per group
	)
	df_all_text_topics_relevance_sentiment_quarter_agg_norm = df_all_text_topics_relevance_sentiment_quarter_agg_norm.groupby(grpby_cols)['sentiment_score'].sum().to_frame()
	df_all_text_topics_relevance_sentiment_quarter_agg_norm2 = df_all_text_topics_relevance_sentiment.groupby(grpby_cols)[subtopic_cols].sum()
	subtopic_sum_srs_abs = df_all_text_topics_relevance_sentiment_quarter_agg_norm2[subtopic_cols].abs().sum(axis=1) # Taking the absolute value here to preserve sign of sentiment score
	df_all_text_topics_relevance_sentiment_quarter_agg_norm2[subtopic_cols] = df_all_text_topics_relevance_sentiment_quarter_agg_norm2[subtopic_cols].div(subtopic_sum_srs_abs, axis=0).fillna(0)  # Normalize the subtopic columns by dividing by the sum of the absolute values of each row, filling NaN with 0
	df_all_text_topics_relevance_sentiment_quarter_agg_norm = df_all_text_topics_relevance_sentiment_quarter_agg_norm.join(df_all_text_topics_relevance_sentiment_quarter_agg_norm2).reset_index()

	# We'll also groupby whether the source is internal or external and append to the aggregated DataFrame
	df_all_text_topics_relevance_sentiment_quarter_agg_norm['source_type'] = "all"
	grpby_cols = ['bank', 'reporting_period', 'risk_category', 'source_type']
	df_all_text_topics_relevance_sentiment_quarter_agg_norm_source = df_all_text_topics_relevance_sentiment.copy()
	df_all_text_topics_relevance_sentiment_quarter_agg_norm_source['sentiment_score'] = df_all_text_topics_relevance_sentiment_quarter_agg_norm_source['sentiment_score'].div(
		df_all_text_topics_relevance_sentiment_quarter_agg_norm_source.groupby(grpby_cols)['sentiment_score'].transform('count'), axis=0 # Normalize the sentiment score by dividing by the number of observations per group
	)
	df_all_text_topics_relevance_sentiment_quarter_agg_norm_source = df_all_text_topics_relevance_sentiment_quarter_agg_norm_source.groupby(grpby_cols)['sentiment_score'].sum().to_frame()
	df_all_text_topics_relevance_sentiment_quarter_agg_norm_source2 = df_all_text_topics_relevance_sentiment.groupby(grpby_cols)[subtopic_cols].sum()
	subtopic_sum_srs_abs_source = df_all_text_topics_relevance_sentiment_quarter_agg_norm_source2[subtopic_cols].abs().sum(axis=1) # Taking the absolute value here to preserve sign of sentiment score
	df_all_text_topics_relevance_sentiment_quarter_agg_norm_source2[subtopic_cols] = df_all_text_topics_relevance_sentiment_quarter_agg_norm_source2[subtopic_cols].div(subtopic_sum_srs_abs_source, axis=0).fillna(0)  # Normalize the subtopic columns by dividing by the sum of the absolute values of each row, filling NaN with 0
	df_all_text_topics_relevance_sentiment_quarter_agg_norm_source = df_all_text_topics_relevance_sentiment_quarter_agg_norm_source.join(
		df_all_text_topics_relevance_sentiment_quarter_agg_norm_source2).reset_index()
	# Append the source type aggregated DataFrame to the main aggregated DataFrame
	df_all_text_topics_relevance_sentiment_quarter_agg_norm = pd.concat(
		[df_all_text_topics_relevance_sentiment_quarter_agg_norm, df_all_text_topics_relevance_sentiment_quarter_agg_norm_source],
		ignore_index=True
	)


	# Save the aggregated DataFrame to a csv and parquet
	fname="multi_topic_modelling_with_relevance_sentiment_quarter_agg_norm"
	fpath_parquet = os.path.join(APP_DATA_FOLDER_PATH, fname + ".parquet")
	fpath_csv = os.path.join(APP_DATA_FOLDER_PATH, fname + ".csv")
	df_all_text_topics_relevance_sentiment_quarter_agg_norm.to_parquet(fpath_parquet)
	df_all_text_topics_relevance_sentiment_quarter_agg_norm.to_csv(fpath_csv, index=False)

else:
	# Load the DataFrame with topic relevance from the parquet file
	fpath_parquet = os.path.join(APP_DATA_FOLDER_PATH, "multi_topic_modelling_with_relevance.parquet")
	df_all_text_topics_relevance = pd.read_parquet(fpath_parquet)

	# Load the DataFrame with topic relevance from the parquet file
	fpath_parquet = os.path.join(APP_DATA_FOLDER_PATH, "multi_topic_modelling_with_relevance_norm.parquet")
	df_all_text_topics_relevance_norm = pd.read_parquet(fpath_parquet)

	# Load the aggregated DataFrame with topic relevance from the parquet file
	fpath_parquet = os.path.join(APP_DATA_FOLDER_PATH, "multi_topic_modelling_with_relevance_quarter_agg_norm.parquet")
	df_all_text_topics_relevance_quarter_agg_norm = pd.read_parquet(fpath_parquet)

	# Load the DataFrame with sentiment weighting from the parquet file
	fpath_parquet = os.path.join(APP_DATA_FOLDER_PATH, "multi_topic_modelling_with_relevance_sentiment_norm.parquet")
	df_all_text_topics_relevance_sentiment_norm = pd.read_parquet(fpath_parquet)

	# Load the aggregated DataFrame with sentiment weighting from the parquet file
	fpath_parquet = os.path.join(APP_DATA_FOLDER_PATH, "multi_topic_modelling_with_relevance_sentiment_quarter_agg_norm.parquet")
	df_all_text_topics_relevance_sentiment_quarter_agg_norm = pd.read_parquet(fpath_parquet)






#### **Q & A Analysis**

In [46]:
# Create a Pydantic model for the topic label
class QandALabel(BaseModel):
	q_and_a_label: Literal['Question', 'Answer', 'Other']

In [47]:
def _make_q_and_a_labelling_prompt(chunk, role):
    """Define a prompt for the Q & A labelling task."""
    return (
        "You are a financial analyst. Classify the following text chunk from a financial earnings transcript as either:\n"
        "- Question: A question asked by an external participant (e.g. analyst or investor)\n"
        "- Answer: A firm representative answering a question\n"
        "- Other: Any other remarks, like greetings or transitions\n"
        "Only classify something as a Question if it's asked by an analyst or external participant.\n\n"
        "Here's the chunk:\n"
        f"{role}: {chunk.strip()}"
        # "Examples:\n"
        # "Role: Analyst\n"
        # "Text: Can you explain the increase in net interest income this quarter?\n"
        # "--> Label: Question\n\n"
        # "Role: CFO\n"
        # "Text: Yes, that's mainly driven by higher rates and repricing of deposits.\n"
        # "--> Label: Answer\n\n"
        # "Role: Operator\n"
        # "Text: Next up, we have James from Barclays.\n"
        # "--> Label: Other\n\n"
        # f"Role: {role}\n"
        # f"Text: {chunk.strip()}\n"
        # "--> Label:"
    )

In [48]:
def label_question_answer_with_llm(chunk, role, llm_backend, llm_model_name):
	"""Label a topic using the LLM."""
	prompt = _make_q_and_a_labelling_prompt(chunk, role)
	labelling_llm = QuestionAnswerTaggingLLM(
		q_and_a_tagging_prompt=prompt,
		response_schema=QandALabel,
		backend=llm_backend,
		model_name=llm_model_name,
	)
	output = labelling_llm.invoke()
	label = output.q_and_a_label
	print(f"{role}: {chunk.strip()} --> Label: {label}")
	return label

In [51]:
if RERUN_Q_AND_A_TAGGING:
	df_q_and_a_all = df_all_text_topics_relevance.copy()
	df_q_and_a_all = df_q_and_a_all.loc[(df_q_and_a_all['section'] == "Q and A")].reset_index(drop=True)
	# Check document type is all "transcript"
	assert (df_q_and_a_all['document_type'] == "transcript").all(), "Expected transcript for all entries"

	# Drop rows where the host or "other" is speaking
	df_q_and_a_all = df_q_and_a_all[~df_q_and_a_all['role'].isin(['Host', 'Other'])].copy()
	df_q_and_a_all.reset_index(drop=True)

	# Initialize a dataframe to hold topic labels
	q_and_a_labels_srs = pd.Series(index=df_q_and_a_all.index)

	# Create a ThreadPoolExecutor to process docs concurrently
	with ThreadPoolExecutor(max_workers=50) as executor:
		futures = {}
		for row_idx, chunk, role in df_q_and_a_all[['text','role']].itertuples():
			future = executor.submit(
				label_question_answer_with_llm,
				chunk=chunk,
				role=role,
				# llm_backend="gemini",
				# llm_model_name="gemini-2.5-flash-preview-05-20"
				llm_backend="openai",
				llm_model_name="gpt-4.1-mini"
			)
			futures[future] = row_idx

		# Wait for all futures to complete
		for future in as_completed(futures):
			try:
				row_idx = futures[future]
				label = future.result()  # This will raise an exception if the processing failed
				q_and_a_labels_srs.loc[row_idx] = label
			except Exception as e:
				print(f"Error processing {futures[future]}: {e}")

	# Get index of section column
	df_q_and_a_all['q_and_a_label'] = q_and_a_labels_srs
	# section_col_idx = df_q_and_a_all.columns.get_loc('section')
	# df_q_and_a_all.insert(section_col_idx+1, 'q_and_a_label', q_and_a_labels_srs)

	for grp_idx, df_q_and_a in df_q_and_a_all.groupby(['bank', 'reporting_period']):
		bank, reporting_period = grp_idx
		# Initialize
		q_and_a_block_tag = []
		q_and_a_risk_categories = []
		q_and_a_topic_labels = []
		previous_tag = 0
		previous_was_analyst_question = False
		num_consecutive_analyst_statements = 0
		for idx, row in df_q_and_a.iterrows():
			role = row['role']
			q_and_a_category = row['q_and_a_label']
			risk_category = row['risk_category']
			topic_label = row['post_merge_topic_label']
			if (role == "Analyst") and (q_and_a_category == "Question") and not previous_was_analyst_question:
				previous_tag += 1
				if num_consecutive_analyst_statements > 0:
					# If the previous statements were from an analyst, we'll reassign those e.g. could have been categorised as Other but are actually the context for the question in the current row
					for i in range(1, num_consecutive_analyst_statements + 1):
						q_and_a_block_tag[-i] = previous_tag
						q_and_a_risk_categories[-i] = risk_category
						q_and_a_topic_labels[-i] = topic_label
				q_and_a_block_tag.append(previous_tag)
				q_and_a_risk_categories.append(risk_category)
				q_and_a_topic_labels.append(topic_label)
				num_consecutive_analyst_statements += 1
				previous_was_analyst_question = True
			else:
				if role == "Analyst":
					num_consecutive_analyst_statements += 1
				else:
					num_consecutive_analyst_statements = 0
				q_and_a_block_tag.append(pd.NA)
				q_and_a_risk_categories.append(pd.NA)
				q_and_a_topic_labels.append(pd.NA)
				previous_was_analyst_question = False

		q_and_a_block_srs = pd.Series(q_and_a_block_tag, index=df_q_and_a.index).ffill()
		q_and_a_topic_labels_srs = pd.Series(q_and_a_topic_labels, index=df_q_and_a.index).ffill()
		q_and_a_risk_categories = pd.Series(q_and_a_risk_categories, index=df_q_and_a.index).ffill()
		sel_bool = ((df_q_and_a_all['bank'] == bank) & (df_q_and_a_all['reporting_period'] == reporting_period))
		df_q_and_a_all.loc[sel_bool, 'q_and_a_block'] = q_and_a_block_srs
		df_q_and_a_all.loc[sel_bool, 'q_and_a_risk_category'] = q_and_a_risk_categories
		df_q_and_a_all.loc[sel_bool, 'q_and_a_topic_label'] = q_and_a_topic_labels_srs

	# Save the DataFrame with Q & A labels to a csv file
	df_q_and_a_all.to_csv("q_and_a_tagged.csv", index=False)

	# df_q_and_a_all = pd.read_csv("q_and_a_tagged.csv")

	# Aggregate within each Q & A block
	df_q_and_a_all_agg = df_q_and_a_all.copy()
	df_q_and_a_all_agg = df_q_and_a_all_agg.dropna(subset=['q_and_a_block']).reset_index(drop=True)
	updated_text_srs = pd.Series(index=df_q_and_a_all_agg.index)
	for grp_idx, df in df_q_and_a_all_agg.groupby(['bank', 'reporting_period', 'q_and_a_block']):
		bank, reporting_period, q_and_a_block = grp_idx
		previous_speaker = ""
		for row_idx, row in df.iterrows():
			speaker = row['source']
			role = row['role']
			text = row['text']
			if speaker != previous_speaker:
				updated_text_srs.loc[row_idx] = f"{speaker} ({role}): {text}\n"
			else:
				updated_text_srs.loc[row_idx] = f"{text}\n"
			previous_speaker = speaker

	df_q_and_a_all_agg['text'] = updated_text_srs

	first_cols = ['page', 'section', 'reporting_period', 'publication_date', 'bank', 'document_type', 'q_and_a_risk_category', 'q_and_a_topic_label']
	first_subtopic_col = df_q_and_a_all_agg.columns.get_loc('risk_category') + 1
	first_q_and_a_label_col = df_q_and_a_all_agg.columns.get_loc('q_and_a_label')
	topic_cols = df_q_and_a_all_agg.columns[first_subtopic_col:first_q_and_a_label_col].tolist()
	sum_cols = ['text'] + topic_cols
	grp_cols = ['bank', 'reporting_period', 'q_and_a_block']

	agg_dict = {col: "first" for col in first_cols}
	agg_dict.update({col: "sum" for col in sum_cols})

	df_q_and_a_all_agg = df_q_and_a_all_agg.groupby(grp_cols).agg(agg_dict)[['text'] + first_cols + topic_cols].reset_index(drop=True)

	# Add a column for source
	reporting_period_split = df_q_and_a_all_agg['reporting_period'].str.split("Q")
	reporting_period_clean = "Q" + reporting_period_split.str.get(1) + ", " + reporting_period_split.str.get(0)
	source_srs = df_q_and_a_all_agg['bank'] + ", " + reporting_period_clean + " Earnings Call Transcript, Page " + df_q_and_a_all_agg['page'].astype(str)
	# Get document_type col index
	document_type_col_idx = df_q_and_a_all_agg.columns.get_loc('document_type')
	# Insert the source column after the document_type column
	df_q_and_a_all_agg.insert(document_type_col_idx + 1, 'source', source_srs)
	# Save the aggregated DataFrame with Q & A labels to a csv file
	df_q_and_a_all_agg.to_csv("q_and_a_aggregated.csv", index=False)

else:
	# Load the DataFrame with Q & A labels from the csv file
	df_q_and_a_all = pd.read_csv("q_and_a_tagged.csv")
	# Load the aggregated DataFrame with Q & A labels from the csv file
	df_q_and_a_all_agg = pd.read_csv("q_and_a_aggregated.csv")



In [57]:
# Create a Pydantic model for the evasiveness score
class EvasivenessScore(BaseModel):
	evasiveness_score: int = Field(...,ge=1,le=5)
	justification: str

In [58]:
def _make_q_and_a_evasiveness_scoring_prompt(q_and_a_text):
    """Define a prompt for evasiveness scoring task."""
    return (
        "You are a financial communication analyst. You are analyzing the clarity and directness of responses in earnings call transcripts of bank firms.\n"
        "Below is a Q&A block from an earnings call. Your task is to:\n"
        "Assess the **evasion level** of the answer(s) provided by management or executives in response to the analyst's question(s).\n"
		"Use the following 5-point scale to rate **evasion**:\n"
		"- **1 (Very Direct):** The answer is clear, specific, and fully addresses the question.\n"
		"- **2 (Mostly Direct):** The answer mostly addresses the question, with some minor vagueness or deflection.\n"
		"- **3 (Neutral):** The answer is somewhat vague or generic, and only partially addresses the question.\n"
		"- **4 (Evasive):** The answer avoids the question with general statements or redirects.\n"
		"- **5 (Very Evasive):** The answer clearly avoids the question, changes topic, or provides no relevant information.\n"
        "In addition to the numeric score, provide a brief explanation (1–3 sentences) justifying why the answer was rated at that level. Be specific about how the response addressed or avoided the question.\n\n"
        f"Here is the Q&A block:\n"
        f"{q_and_a_text.strip()}"
    )

In [59]:
def assign_evasiveness_score_with_llm(q_and_a_text, llm_backend, llm_model_name):
	"""Label a topic using the LLM."""
	prompt = _make_q_and_a_evasiveness_scoring_prompt(q_and_a_text)
	evasiveness_tagging_llm = EvasivenessTaggingLLM(
		evasiveness_tagging_prompt=prompt,
		response_schema=EvasivenessScore,
		backend=llm_backend,
		model_name=llm_model_name,
	)
	output = evasiveness_tagging_llm.invoke()
	# Print the output
	print("---------------------------------------")
	print(q_and_a_text.strip())
	print(f"Evasiveness Score: {output.evasiveness_score}, Justification: {output.justification}")
	return {
		"evasiveness_score": output.evasiveness_score,
		"justification": output.justification
	}

In [60]:
if RERUN_EVASIVE_ANSWER_DETECTION:
	# Initialize a dataframe to hold topic labels
	evasiveness_score_df = pd.DataFrame(index=df_q_and_a_all_agg.index, columns=['evasiveness_score', 'justification'])
	# Create a ThreadPoolExecutor to process docs concurrently
	with ThreadPoolExecutor(max_workers=25) as executor:
		futures = {}
		for row_idx, q_and_a_text in df_q_and_a_all_agg['text'].items():
			future = executor.submit(
				assign_evasiveness_score_with_llm,
				q_and_a_text=q_and_a_text,
				# llm_backend="gemini",
				# llm_model_name="gemini-2.5-flash-preview-05-20"
				llm_backend="openai",
				llm_model_name="gpt-4.1-mini"
			)
			futures[future] = row_idx

		# Wait for all futures to complete
		for future in as_completed(futures):
			try:
				row_idx = futures[future]
				res = future.result()  # This will raise an exception if the processing failed
				evasiveness_score = res['evasiveness_score']
				justification = res['justification']
				# Add the results to the DataFrame
				evasiveness_score_df.loc[row_idx, 'evasiveness_score'] = evasiveness_score
				evasiveness_score_df.loc[row_idx, 'justification'] = justification
			except Exception as e:
				print(f"Error processing {futures[future]}: {e}")

	# Get index of section column
	df_q_and_a_all_agg_evasiveness = df_q_and_a_all_agg.join(evasiveness_score_df, how='left')

	# Save the DataFrame with evasiveness scores
	fname="q_and_a_evasiveness_scores"
	fpath_parquet = os.path.join(APP_DATA_FOLDER_PATH, fname + ".parquet")
	fpath_csv = os.path.join(APP_DATA_FOLDER_PATH, fname + ".csv")
	df_q_and_a_all_agg_evasiveness.to_parquet(fpath_parquet)
	df_q_and_a_all_agg_evasiveness.to_csv(fpath_csv, index=False)

else:
	# Load the DataFrame with evasiveness scores
	fpath_parquet = os.path.join(APP_DATA_FOLDER_PATH, "q_and_a_evasiveness_scores.parquet")
	df_q_and_a_all_agg_evasiveness = pd.read_parquet(fpath_parquet)



---------------------------------------
Vivek Juneja (Analyst): One more, Alastair. Office Commercial Real Estate, you gave the geographical mix in your slides for the total Commercial Real Estate portfolio. Can you give us some similar thing for the office Commercial Real Estate portfolio?
Alastair Borthwick (CFO): But I don't think you're going to find anything there other than sort of typical geographic distribution similar to the way we serve our customers around the United States.
Evasiveness Score: 4, Justification: The CFO did not provide specific geographic details for the office Commercial Real Estate portfolio as requested. Instead, he gave a general statement about typical geographic distribution similar to their overall customer base, which avoids directly answering the question with precise data.
---------------------------------------
Gerard Cassidy (Analyst): Okay. Actually, Brian and Alastair, you guys obviously have been through a few cycles. Why is commercial so stron

#### **VISUALISATIONS**

In [62]:
fig = px.colors.qualitative.swatches()
fig.show()

In [63]:
colours_list_g10 = px.colors.qualitative.G10

In [64]:
def generate_multiline_chart(df, x_data_col, y_data_cols, x_title, y_title, plot_title=None, marker_colors=colours_list_g10, marker_size=1, line_colors=colours_list_g10, line_legend_labels=None):

	assert len(y_data_cols) <= len(line_colors), "Number of y_data_cols must not exceed number of line_colors"

	fig = go.Figure()

	for i, y_data_col in enumerate(y_data_cols):
		fig.add_trace(
			go.Scatter(
				x=df[x_data_col],
				y=df[y_data_col],
				mode='lines+markers',
				marker=dict(
					color=marker_colors[i],
					size=marker_size,
				),
				line=dict(color=line_colors[i]),
				name=line_legend_labels[i],
				showlegend=True,
				yaxis='y')
		)

	fig.update_layout(
		title=plot_title,
		xaxis=dict(title=x_title),
		yaxis=dict(title=y_title),
	)

	fig.update_layout(
		legend=dict(
			xanchor="center",
			x=0.5,
			yanchor="bottom",
			y=-0.5,
			orientation='h',
		)
	)
	return fig

In [67]:
# df_all_text_topics_relevance_quarter_agg# Load the DataFrame with topic relevance from the parquet file
# fpath_parquet = os.path.join(APP_DATA_FOLDER_PATH, "multi_topic_modelling_with_relevance_norm.parquet")
# df_all_text_topics_relevance_norm = pd.read_parquet(fpath_parquet)

# # Load the aggregated DataFrame with topic relevance from the parquet file
# fpath_parquet = os.path.join(APP_DATA_FOLDER_PATH, "multi_topic_modelling_with_relevance_quarter_agg_norm.parquet")
# df_all_text_topics_relevance_quarter_agg_norm = pd.read_parquet(fpath_parquet)

# # Load the DataFrame with sentiment weighting from the parquet file
# fpath_parquet = os.path.join(APP_DATA_FOLDER_PATH, "multi_topic_modelling_with_relevance_sentiment_norm.parquet")
# df_all_text_topics_relevance_sentiment_norm = pd.read_parquet(fpath_parquet)

# # Load the aggregated DataFrame with sentiment weighting from the parquet file
# fpath_parquet = os.path.join(APP_DATA_FOLDER_PATH, "multi_topic_modelling_with_relevance_sentiment_quarter_agg_norm.parquet")
# df_all_text_topics_relevance_sentiment_quarter_agg_norm = pd.read_parquet(fpath_parquet)

In [69]:
df_all_text_topics_relevance_quarter_agg_norm

,bank,reporting_period,risk_category,risk_proportion,Macroeconomic Outlook,U.S. Retail Banking,Operational Transformation,Global Banking Strategy,Asset Liability Management,Interest Rate Risk,...,Return Ratios,Trading Assets and Value at Risk,Portfolio Maturities and Balances,Capital and Liquidity,Treasury Trade Solutions Loans,Financial Institution Trends,US Dollar Clearing Volume Trends,Equity Trend,Diluted Shares Trend,source_type
0,Bank of America,2023Q1,Asset Quality and Credit Risk,0.149834,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.002234,0.012381,0.000000,0.0,0.008823,all
1,Bank of America,2023Q1,Capital Adequacy,0.075472,0.000000,0.000000,0.000000,0.002710,0.000000,0.000000,...,0.000000,0.0,0.005498,0.012955,0.000000,0.000000,0.000000,0.0,0.062150,all
2,Bank of America,2023Q1,ESG and Reputation Risk,0.009989,0.000000,0.166499,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.064171,0.000000,0.0,0.000000,all
3,Bank of America,2023Q1,Interest Rate Risk,0.048835,0.000000,0.000000,0.005811,0.012198,0.000000,0.013118,...,0.000000,0.0,0.044819,0.007607,0.000000,0.014209,0.000000,0.0,0.000000,all
4,Bank of America,2023Q1,Liquidity Risk,0.129856,0.000000,0.011216,0.000000,0.013459,0.001372,0.000000,...,0.000000,0.0,0.018010,0.035280,0.001384,0.010603,0.002901,0.0,0.007453,all
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,JPMorgan,2025Q1,Profitability,0.330667,0.000866,0.000728,0.000000,0.000765,0.000000,0.000000,...,0.002444,0.0,0.000000,0.000000,0.000000,0.000850,0.000000,0.0,0.001912,internal
673,JPMorgan,2025Q1,Regulatory & Compliance Risk,0.029333,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,external
674,JPMorgan,2025Q1,Regulatory & Compliance Risk,0.021333,0.044457,0.000000,0.000000,0.046972,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.096535,0.000000,0.0,0.000000,internal
675,JPMorgan,2025Q1,Strategic and Business Model Risk,0.053333,0.000000,0.024385,0.000000,0.026208,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.063121,0.000000,0.0,0.000000,external


In [90]:
risk_category = "Liquidity Risk"
source_type = "internal"
sel_bool = ((df_all_text_topics_relevance_quarter_agg_norm['source_type'] == source_type) & (df_all_text_topics_relevance_quarter_agg_norm['risk_category'] == risk_category))
df_topics_relevance_quarter_plotting = df_all_text_topics_relevance_quarter_agg_norm[sel_bool].copy()
df_topics_relevance_quarter_plotting.drop(columns=['risk_category', 'source_type'], inplace=True)
df_topics_relevance_quarter_plotting.rename(columns={"risk_proportion": f"Risk Prevalence: {risk_category}"}, inplace=True)
df_topics_relevance_quarter_plotting = df_topics_relevance_quarter_plotting.pivot(
	columns='bank',
	index='reporting_period',
)
df_topics_relevance_quarter_plotting

Risk Prevalence: Liquidity Risk                      \
bank                             Bank of America Citigroup  JPMorgan   
reporting_period                                                       
2023Q1                                  0.129856  0.130246  0.088832   
2023Q2                                  0.107792  0.076401  0.102564   
2023Q3                                  0.096186  0.069124  0.074689   
2023Q4                                  0.094118  0.048276  0.073276   
2024Q1                                  0.100977  0.075472  0.085561   
2024Q2                                  0.105263  0.062102  0.081720   
2024Q3                                  0.080000  0.030504  0.101990   
2024Q4                                  0.083478  0.014388  0.040498   
2025Q1                                  0.058065  0.053254  0.058667   

                 Macroeconomic Outlook                      \
bank                   Bank of America Citigroup  JPMorgan   
reporting_period                                             
2023Q1                         0.00000       0.0  0.005375   
2023Q2                         0.00235       0.0  0.000000   
2023Q3                         0.00000       0.0  0.000000   
2023Q4                         0.00247       0.0  0.000000   
2024Q1                         0.00000       0.0  0.006708   
2024Q2                         0.00000       0.0  0.000000   
2024Q3                         0.00000       0.0  0.000000   
2024Q4                         0.00000       0.0  0.000000   
2025Q1                         0.00000       0.0  0.000000   

                 U.S. Retail Banking                      \
bank                 Bank of America Citigroup  JPMorgan   
reporting_period                                           
2023Q1                      0.011216  0.018827  0.000000   
2023Q2                      0.013631  0.016043  0.004228   
2023Q3                      0.014866  0.014151  0.000000   
2023Q4                      0.014682  0.016387  0.008355   
2024Q1                      0.014617  0.013846  0.000000   
2024Q2                      0.019202  0.015279  0.000000   
2024Q3                      0.015190  0.022657  0.000000   
2024Q4                      0.027956  0.027499  0.000000   
2025Q1                      0.019238  0.010334  0.000000   

                 Operational Transformation  ... Financial Institution Trends  \
bank                        Bank of America  ...                     JPMorgan   
reporting_period                             ...                                
2023Q1                             0.000000  ...                     0.009863   
2023Q2                             0.000000  ...                     0.004405   
2023Q3                             0.000000  ...                     0.004747   
2023Q4                             0.000000  ...                     0.026946   
2024Q1                             0.000000  ...                     0.004930   
2024Q2                             0.003318  ...                     0.015454   
2024Q3                             0.000000  ...                     0.000000   
2024Q4                             0.000000  ...                     0.024774   
2025Q1                             0.004569  ...                     0.035386   

                 US Dollar Clearing Volume Trends                      \
bank                              Bank of America Citigroup  JPMorgan   
reporting_period                                                        
2023Q1                                   0.002901  0.005096  0.000000   
2023Q2                                   0.002048  0.000000  0.009625   
2023Q3                                   0.002613  0.000000  0.004862   
2023Q4                                   0.006586  0.019377  0.009442   
2024Q1                                   0.005463  0.011995  0.000000   
2024Q2                                   0.003262  0.019795  0.003279   
2024Q3                                   0.006687  0.000

In [91]:
df_topics_relevance_quarter_plotting.columns = [', '.join(col).strip() for col in df_topics_relevance_quarter_plotting.columns.values]
df_topics_relevance_quarter_plotting.reset_index(inplace=True)
df_topics_relevance_quarter_plotting

,reporting_period,"Risk Prevalence: Liquidity Risk, Bank of America","Risk Prevalence: Liquidity Risk, Citigroup","Risk Prevalence: Liquidity Risk, JPMorgan","Macroeconomic Outlook, Bank of America","Macroeconomic Outlook, Citigroup","Macroeconomic Outlook, JPMorgan","U.S. Retail Banking, Bank of America","U.S. Retail Banking, Citigroup","U.S. Retail Banking, JPMorgan",...,"Financial Institution Trends, JPMorgan","US Dollar Clearing Volume Trends, Bank of America","US Dollar Clearing Volume Trends, Citigroup","US Dollar Clearing Volume Trends, JPMorgan","Equity Trend, Bank of America","Equity Trend, Citigroup","Equity Trend, JPMorgan","Diluted Shares Trend, Bank of America","Diluted Shares Trend, Citigroup","Diluted Shares Trend, JPMorgan"
0,2023Q1,0.129856,0.130246,0.088832,0.00000,0.0,0.005375,0.011216,0.018827,0.000000,...,0.009863,0.002901,0.005096,0.000000,0.0,0.0,0.0,0.007453,0.0,0.0
1,2023Q2,0.107792,0.076401,0.102564,0.00235,0.0,0.000000,0.013631,0.016043,0.004228,...,0.004405,0.002048,0.000000,0.009625,0.0,0.0,0.0,0.000000,0.0,0.0
2,2023Q3,0.096186,0.069124,0.074689,0.00000,0.0,0.000000,0.014866,0.014151,0.000000,...,0.004747,0.002613,0.000000,0.004862,0.0,0.0,0.0,0.000000,0.0,0.0
3,2023Q4,0.094118,0.048276,0.073276,0.00247,0.0,0.000000,0.014682,0.016387,0.008355,...,0.026946,0.006586,0.019377,0.009442,0.0,0.0,0.0,0.000000,0.0,0.0
4,2024Q1,0.100977,0.075472,0.085561,0.00000,0.0,0.006708,0.014617,0.013846,0.000000,...,0.004930,0.005463,0.011995,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
5,2024Q2,0.105263,0.062102,0.081720,0.00000,0.0,0.000000,0.019202,0.015279,0.000000,...,0.015454,0.003262,0.019795,0.003279,0.0,0.0,0.0,0.000000,0.0,0.0
6,2024Q3,0.080000,0.030504,0.101990,0.00000,0.0,0.000000,0.015190,0.022657,0.000000,...,0.000000,0.006687,0.000000,0.004877,0.0,0.0,0.0,0.000000,0.0,0.0
7,2024Q4,0.083478,0.014388,0.040498,0.00000,0.0,0.000000,0.027956,0.027499,0.000000,...,0.024774,0.003477,0.000000,0.011888,0.0,0.0,0.0,0.003668,0.0,0.0
8,2025Q1,0.058065,0.053254,0.058667,0.00000,0.0,0.000000,0.019238,0.010334,0.000000,...,0.035386,0.000000,0.007395,0.008421,0.0,0.0,0.0,0.000000,0.0,0.0


In [92]:
df_topics_relevance_quarter_plotting.iloc[:,4:].sum(axis=1)

0    3.000001
1    2.999999
2    3.000001
3    3.000000
4    3.000000
5    2.999999
6    3.000000
7    3.000000
8    2.999999
dtype: float32

In [105]:
risk_category_cols = [col for col in df_topics_relevance_quarter_plotting.columns if col.startswith("Risk Prevalence")]
topic_cols = [col for col in df_topics_relevance_quarter_plotting.columns if col not in ['reporting_period'] + risk_category_cols]

In [106]:
# Generate a multiline chart for the topic relevance scores
cols_to_plot = [col for col in risk_category_cols if risk_category in col]
legend_labels = [col.split(",")[-1].strip() for col in cols_to_plot]

fig_topics_prevalence_quarter = generate_multiline_chart(
	df=df_topics_relevance_quarter_plotting,
	x_data_col='reporting_period',
	y_data_cols=cols_to_plot,
	x_title='Reporting Period',
	y_title='Risk Prevalence Score (Normalized)',
	plot_title=f"Risk Prevalence Trends: {risk_category}",
	marker_size=5,
	line_legend_labels=legend_labels,
)

fig_topics_prevalence_quarter.show()

In [108]:
# Generate a multiline chart for the topic relevance scores
subtopic = "Liquidity Coverage Ratio"
cols_to_plot = [col for col in subtopic_cols if subtopic in col]
legend_labels = [col.split(",")[-1].strip() for col in cols_to_plot]

fig_topics_relevance_quarter = generate_multiline_chart(
	df=df_topics_relevance_quarter_plotting,
	x_data_col='reporting_period',
	y_data_cols=cols_to_plot,
	x_title='Reporting Period',
	y_title='Topic Prevalence Score (Normalized)',
	plot_title=f"Topic Prevalence Trends: {subtopic}",
	marker_size=5,
	line_legend_labels=legend_labels,
)

fig_topics_relevance_quarter.show()

In [119]:
df_topics_relevance_quarter_plotting[[col for col in df_topics_relevance_quarter_plotting.columns if col.startswith("Bank Performance")]]

,"Bank Performance, Bank of America","Bank Performance, Citigroup","Bank Performance, JPMorgan"
0,0.027382,0.048743,0.048945
1,0.030338,0.024545,0.032845
2,0.014270,0.047959,0.047841
3,0.024063,0.083711,0.065862
4,0.032817,0.043171,0.033462
5,0.031902,0.014970,0.043630
6,0.024825,0.091539,0.038586
7,0.016404,0.000000,0.089499
8,0.058097,0.087760,0.049748


In [125]:
def get_diverging_topics(df, topics, top_k=10, quarter=None, lookback=0):
	if df['reporting_period'].duplicated().any():
		raise ValueError("The 'reporting_period' column must not have duplicate values.")
	quarters_srs = df['reporting_period']
	if not quarter:
		quarter = quarters_srs.iloc[-1]  # Get the most recent quarter if not specified
	last_quarter_idx = quarters_srs[quarters_srs == quarter].index[0]
	quarters_to_consider = quarters_srs.loc[last_quarter_idx - lookback:last_quarter_idx + lookback + 1].tolist()
	# Filter the DataFrame for the specified quarters
	df_filtered = df[df['reporting_period'].isin(quarters_to_consider)].copy()
	# We work out the average spread of the topics across the specified quarters for each topic
	topic_srs = pd.Series(index=topics, dtype=float)
	for topic in topics:
		topic_cols = [col for col in df_filtered.columns if col.split(",")[0].strip() == topic]
		topic_srs.loc[topic] = df_filtered[topic_cols].var(axis=1).mean()

	# Sort the topics by their average spread
	topic_srs = topic_srs.sort_values(ascending=False)

	# Get the top k topics with the highest average spread
	top_k_topics = topic_srs.head(top_k).index.tolist()

	return top_k_topics




def get_trending_topics(df, topics, top_k=10, trend_type="up", quarter=None, lookback=1):
	if df['reporting_period'].duplicated().any():
		raise ValueError("The 'reporting_period' column must not have duplicate values.")
	quarters_srs = df['reporting_period']
	if not quarter:
		quarter = quarters_srs.iloc[-1]  # Get the most recent quarter if not specified
	last_quarter_idx = quarters_srs[quarters_srs == quarter].index[0]
	quarters_to_consider = quarters_srs.loc[last_quarter_idx - lookback:last_quarter_idx + lookback + 1].tolist()
	# Filter the DataFrame for the specified quarters
	df_filtered = df[df['reporting_period'].isin(quarters_to_consider)].copy()
	# We work out the max  of the topics across the specified quarters for each topic
	topic_srs = pd.Series(index=topics, dtype=float)
	for topic in topics:
		topic_cols = [col for col in df_filtered.columns if col.split(",")[0].strip() == topic]
		slopes = []
		for topic_col in topic_cols:
			# Calculate the slope of the topic column over the specified quarters
			slope = df_filtered[topic_col].diff().mean()
			slopes.append(slope)
		if trend_type == "up":
			max_slope = max(slopes)
			topic_srs.loc[topic] = max_slope
		else:
			min_slope = min(slopes)
			topic_srs.loc[topic] = min_slope

	# Sort the topics by their max slope
	if trend_type == "up":
		topic_srs = topic_srs.sort_values(ascending=False)
	else:
		topic_srs = topic_srs.sort_values(ascending=True)

	# Get the top k topics with the highest average spread
	top_k_topics = topic_srs.head(top_k).index.tolist()

	return top_k_topics



first_topic = df_all_text_topics_relevance_quarter_agg_norm.columns.get_loc("risk_proportion") + 1
topics = df_all_text_topics_relevance_quarter_agg_norm.columns[first_topic:-1].tolist()
diverging_topics = get_diverging_topics(df=df_topics_relevance_quarter_plotting, topics=topics, top_k=10, lookback=2)

diverging_topic = diverging_topics[0]
cols_to_plot = [col for col in subtopic_cols if col.split(",")[0].strip() == diverging_topic]
legend_labels = [col.split(",")[-1].strip() for col in cols_to_plot]

fig_topics_relevance_quarter = generate_multiline_chart(
	df=df_topics_relevance_quarter_plotting,
	x_data_col='reporting_period',
	y_data_cols=cols_to_plot,
	x_title='Reporting Period',
	y_title='Topic Prevalence Score (Normalized)',
	plot_title=f"Topic Prevalence Trends: {diverging_topic}",
	marker_size=5,
	line_legend_labels=legend_labels,
)

fig_topics_relevance_quarter.show()


In [127]:
uptrending_topics = get_trending_topics(df=df_topics_relevance_quarter_plotting, topics=topics, top_k=10, lookback=1)

uptrending_topic = uptrending_topics[0]
cols_to_plot = [col for col in subtopic_cols if col.split(",")[0].strip() == uptrending_topic]
legend_labels = [col.split(",")[-1].strip() for col in cols_to_plot]

fig_topics_relevance_quarter = generate_multiline_chart(
	df=df_topics_relevance_quarter_plotting,
	x_data_col='reporting_period',
	y_data_cols=cols_to_plot,
	x_title='Reporting Period',
	y_title='Topic Prevalence Score (Normalized)',
	plot_title=f"Topic Prevalence Trends: {uptrending_topic}",
	marker_size=5,
	line_legend_labels=legend_labels,
)

fig_topics_relevance_quarter.show()

In [ ]:
def update_heatmap(df, x_data_col, y_data_col, z_data_col, x_title, y_title, colour_title=None, colour_scale=[COLOURS['Pearly Purple'], COLOURS['Persian Green']]):
    if x_data_col == "UtilisationPriceCategory" or x_data_col == "TimeOfDayCat":
        if x_data_col == "UtilisationPriceCategory":
            x_cats = df.sort_values('Price')[x_data_col].unique().tolist()
        else:
            x_cats = df.sort_values('TimeOfDay')[x_data_col].unique().tolist()
    else:
        x_cats = None

    if y_data_col == "UtilisationPriceCategory" or y_data_col == "TimeOfDayCat":
        if y_data_col == "UtilisationPriceCategory":
            y_cats = df.sort_values('Price')[y_data_col].unique().tolist()
        else:
            y_cats = df.sort_values('TimeOfDay')[y_data_col].unique().tolist()
    else:
        y_cats = None

    df = df.groupby([x_data_col, y_data_col])[z_data_col].mean().reset_index()
    mux = pd.MultiIndex.from_product([df[x_data_col].unique(), df[y_data_col].unique()])
    df = df.set_index([x_data_col, y_data_col]).reindex(mux).reset_index().set_axis(df.columns, axis=1)
    # df = df.sort_values([y_data_col,x_data_col]).reset_index()
    df = pd.pivot(df, index=y_data_col, columns=x_data_col, values=z_data_col)
    if x_cats:
        df.columns = pd.Categorical(df.columns, x_cats, ordered=True)
        df.sort_index(axis=1, inplace=True)
    if y_cats:
        df.index = pd.Categorical(df.index, y_cats, ordered=True)
        df.sort_index(axis=0, inplace=True)
    zd = df.values
    xd = df.columns
    yd = df.index

    fig = go.Figure()

    fig.add_trace(
        go.Heatmap(
            z=zd,
            x=xd,
            y=yd,
            colorbar=dict(title=colour_title),
            colorscale=[[0, colour_scale[0]], [1, colour_scale[1]]],
            showlegend=False,
            yaxis='y')
    )

    fig.update_layout(
        xaxis=dict(title=x_title),
        yaxis=dict(title=y_title),
    )

    fig.update_layout(xaxis_type='category', yaxis_type='category')

    fig.update_xaxes(tickangle=30)
    fig.update_xaxes(title_standoff=200)
    # fig.update_yaxes(tickangle = -30)
    fig.update_yaxes(title_standoff=10)
    fig.update_layout(
        margin=dict(l=250),
    )

    return fig

In [104]:
df_topics_relevance_quarter_plotting

,reporting_period,"Risk Prevalence: Liquidity Risk, Bank of America","Risk Prevalence: Liquidity Risk, Citigroup","Risk Prevalence: Liquidity Risk, JPMorgan","Macroeconomic Outlook, Bank of America","Macroeconomic Outlook, Citigroup","Macroeconomic Outlook, JPMorgan","U.S. Retail Banking, Bank of America","U.S. Retail Banking, Citigroup","U.S. Retail Banking, JPMorgan",...,"Financial Institution Trends, JPMorgan","US Dollar Clearing Volume Trends, Bank of America","US Dollar Clearing Volume Trends, Citigroup","US Dollar Clearing Volume Trends, JPMorgan","Equity Trend, Bank of America","Equity Trend, Citigroup","Equity Trend, JPMorgan","Diluted Shares Trend, Bank of America","Diluted Shares Trend, Citigroup","Diluted Shares Trend, JPMorgan"
0,2023Q1,0.129856,0.130246,0.088832,0.00000,0.0,0.005375,0.011216,0.018827,0.000000,...,0.009863,0.002901,0.005096,0.000000,0.0,0.0,0.0,0.007453,0.0,0.0
1,2023Q2,0.107792,0.076401,0.102564,0.00235,0.0,0.000000,0.013631,0.016043,0.004228,...,0.004405,0.002048,0.000000,0.009625,0.0,0.0,0.0,0.000000,0.0,0.0
2,2023Q3,0.096186,0.069124,0.074689,0.00000,0.0,0.000000,0.014866,0.014151,0.000000,...,0.004747,0.002613,0.000000,0.004862,0.0,0.0,0.0,0.000000,0.0,0.0
3,2023Q4,0.094118,0.048276,0.073276,0.00247,0.0,0.000000,0.014682,0.016387,0.008355,...,0.026946,0.006586,0.019377,0.009442,0.0,0.0,0.0,0.000000,0.0,0.0
4,2024Q1,0.100977,0.075472,0.085561,0.00000,0.0,0.006708,0.014617,0.013846,0.000000,...,0.004930,0.005463,0.011995,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
5,2024Q2,0.105263,0.062102,0.081720,0.00000,0.0,0.000000,0.019202,0.015279,0.000000,...,0.015454,0.003262,0.019795,0.003279,0.0,0.0,0.0,0.000000,0.0,0.0
6,2024Q3,0.080000,0.030504,0.101990,0.00000,0.0,0.000000,0.015190,0.022657,0.000000,...,0.000000,0.006687,0.000000,0.004877,0.0,0.0,0.0,0.000000,0.0,0.0
7,2024Q4,0.083478,0.014388,0.040498,0.00000,0.0,0.000000,0.027956,0.027499,0.000000,...,0.024774,0.003477,0.000000,0.011888,0.0,0.0,0.0,0.003668,0.0,0.0
8,2025Q1,0.058065,0.053254,0.058667,0.00000,0.0,0.000000,0.019238,0.010334,0.000000,...,0.035386,0.000000,0.007395,0.008421,0.0,0.0,0.0,0.000000,0.0,0.0


In [103]:
df_topics_relevance_quarter_plotting

,reporting_period,"Risk Prevalence: Liquidity Risk, Bank of America","Risk Prevalence: Liquidity Risk, Citigroup","Risk Prevalence: Liquidity Risk, JPMorgan","Macroeconomic Outlook, Bank of America","Macroeconomic Outlook, Citigroup","Macroeconomic Outlook, JPMorgan","U.S. Retail Banking, Bank of America","U.S. Retail Banking, Citigroup","U.S. Retail Banking, JPMorgan",...,"Financial Institution Trends, JPMorgan","US Dollar Clearing Volume Trends, Bank of America","US Dollar Clearing Volume Trends, Citigroup","US Dollar Clearing Volume Trends, JPMorgan","Equity Trend, Bank of America","Equity Trend, Citigroup","Equity Trend, JPMorgan","Diluted Shares Trend, Bank of America","Diluted Shares Trend, Citigroup","Diluted Shares Trend, JPMorgan"
0,2023Q1,0.129856,0.130246,0.088832,0.00000,0.0,0.005375,0.011216,0.018827,0.000000,...,0.009863,0.002901,0.005096,0.000000,0.0,0.0,0.0,0.007453,0.0,0.0
1,2023Q2,0.107792,0.076401,0.102564,0.00235,0.0,0.000000,0.013631,0.016043,0.004228,...,0.004405,0.002048,0.000000,0.009625,0.0,0.0,0.0,0.000000,0.0,0.0
2,2023Q3,0.096186,0.069124,0.074689,0.00000,0.0,0.000000,0.014866,0.014151,0.000000,...,0.004747,0.002613,0.000000,0.004862,0.0,0.0,0.0,0.000000,0.0,0.0
3,2023Q4,0.094118,0.048276,0.073276,0.00247,0.0,0.000000,0.014682,0.016387,0.008355,...,0.026946,0.006586,0.019377,0.009442,0.0,0.0,0.0,0.000000,0.0,0.0
4,2024Q1,0.100977,0.075472,0.085561,0.00000,0.0,0.006708,0.014617,0.013846,0.000000,...,0.004930,0.005463,0.011995,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
5,2024Q2,0.105263,0.062102,0.081720,0.00000,0.0,0.000000,0.019202,0.015279,0.000000,...,0.015454,0.003262,0.019795,0.003279,0.0,0.0,0.0,0.000000,0.0,0.0
6,2024Q3,0.080000,0.030504,0.101990,0.00000,0.0,0.000000,0.015190,0.022657,0.000000,...,0.000000,0.006687,0.000000,0.004877,0.0,0.0,0.0,0.000000,0.0,0.0
7,2024Q4,0.083478,0.014388,0.040498,0.00000,0.0,0.000000,0.027956,0.027499,0.000000,...,0.024774,0.003477,0.000000,0.011888,0.0,0.0,0.0,0.003668,0.0,0.0
8,2025Q1,0.058065,0.053254,0.058667,0.00000,0.0,0.000000,0.019238,0.010334,0.000000,...,0.035386,0.000000,0.007395,0.008421,0.0,0.0,0.0,0.000000,0.0,0.0


In [105]:
df_all_text_topics_relevance_sentiment_quarter_agg_norm

,bank,reporting_period,Macroeconomic Outlook,Systemic Stability & Regulation,Business Transformation,Growth Plans,Value At Risk,Interest Rate Sensitivity,Balance Sheet Management,Deposit Base Stability,...,RISK CATEGORY: Capital Adequacy,RISK CATEGORY: ESG and Reputation Risk,RISK CATEGORY: Interest Rate Risk,RISK CATEGORY: Liquidity Risk,RISK CATEGORY: Macroeconomic Risk,RISK CATEGORY: Market and Volatility Risk,RISK CATEGORY: Operational Risk,RISK CATEGORY: Profitability,RISK CATEGORY: Regulatory & Compliance Risk,RISK CATEGORY: Strategic and Business Model Risk
0,Citigroup,2023Q1,0.340327,0.220541,0.977047,0.805073,0.102109,0.133029,0.239783,0.662238,...,1.214962,0.009599,0.339302,0.456059,0.008185,1.088627,0.290626,6.440624,0.021869,0.846877
1,Citigroup,2023Q2,0.035306,-0.175488,2.555608,2.496304,0.000000,0.177733,0.406166,0.593362,...,2.968114,0.025837,0.278528,1.026347,0.254047,0.944383,0.143103,-0.559611,-0.054363,3.020865
2,Citigroup,2023Q3,-0.601743,-0.104564,2.746828,1.441981,0.000000,0.111819,0.000000,0.704157,...,0.710816,0.028586,0.208853,0.183026,-0.076065,0.971564,0.329938,5.572853,-0.006511,1.475966
3,Citigroup,2023Q4,0.317550,0.119368,3.621517,2.876357,-0.014934,0.345359,0.124997,0.120734,...,0.899677,0.019801,-0.146758,0.070311,0.122909,0.138879,0.442716,1.902327,0.020252,3.087349
4,Citigroup,2024Q1,-0.347859,0.000000,4.110513,2.420880,0.317483,0.308822,0.338248,0.301243,...,0.744371,0.037763,0.180227,0.281195,-0.007075,0.898005,0.412533,4.783454,0.038098,2.513278
5,Citigroup,2024Q2,-0.344308,0.302413,4.337381,2.398360,-0.351328,-0.288231,0.339844,0.181482,...,1.883815,0.030172,-0.283517,0.031963,0.027799,1.029296,-0.205209,2.282180,0.102170,2.826735
6,Citigroup,2024Q3,0.338124,0.231008,2.561336,2.781485,0.000000,-0.199259,0.108624,0.499246,...,0.636251,0.017157,0.101179,0.434148,-0.003182,0.909468,0.382246,5.660540,0.053698,1.630081
7,Citigroup,2024Q4,0.200094,0.000000,3.185312,3.039439,0.000000,0.000000,0.000000,0.000000,...,0.823095,0.007632,0.143881,0.164699,0.124372,1.034389,0.336785,6.238587,0.030725,1.670724
8,Citigroup,2025Q1,0.258709,0.207143,1.038792,1.894327,-0.126321,0.000000,0.112733,0.440800,...,0.426012,0.000000,0.357124,0.121502,0.035115,1.050395,0.232925,7.152725,0.043918,1.067563
9,JPMorgan,2023Q1,-2.081631,-0.218864,0.000000,-0.233276,0.000000,-1.718887,0.000000,-1.075892,...,0.235483,0.000000,-0.228586,-0.722118,-0.431869,0.596191,0.197963,7.776062,-0.016603,0.064535


In [106]:
df_topics_relevance_sentiment_quarter_plotting = df_all_text_topics_relevance_sentiment_quarter_agg_norm.pivot(
	columns='bank',
	index='reporting_period',
)
df_topics_relevance_sentiment_quarter_plotting

Macroeconomic Outlook            \
bank                         Citigroup  JPMorgan   
reporting_period                                   
2023Q1                        0.340327 -2.081631   
2023Q2                        0.035306 -0.051790   
2023Q3                       -0.601743 -2.897782   
2023Q4                        0.317550  0.018854   
2024Q1                       -0.347859  0.116909   
2024Q2                       -0.344308 -0.399786   
2024Q3                        0.338124  0.346621   
2024Q4                        0.200094 -0.176315   
2025Q1                        0.258709  0.456509   

                 Systemic Stability & Regulation            \
bank                                   Citigroup  JPMorgan   
reporting_period                                             
2023Q1                                  0.220541 -0.218864   
2023Q2                                 -0.175488 -0.987473   
2023Q3                                 -0.104564 -0.574525   
2023Q4                                  0.119368  0.000000   
2024Q1                                  0.000000  0.000000   
2024Q2                                  0.302413  0.000000   
2024Q3                                  0.231008  0.298294   
2024Q4                                  0.000000  0.000000   
2025Q1                                  0.207143 -0.241117   

                 Business Transformation           Growth Plans            \
bank                           Citigroup  JPMorgan    Citigroup  JPMorgan   
reporting_period                                                            
2023Q1                          0.977047  0.000000     0.805073 -0.233276   
2023Q2                          2.555608  0.008517     2.496304  1.821243   
2023Q3                          2.746828  0.340243     1.441981 -0.012745   
2023Q4                          3.621517  0.000000     2.876357  0.791051   
2024Q1                          4.110513  0.000000     2.420880  0.846485   
2024Q2                          4.337381  0.000000     2.398360  0.322455   
2024Q3                          2.561336  0.000000     2.781485  0.629683   
2024Q4                          3.185312  0.382413     3.039439  0.389032   
2025Q1                          1.038792  0.285707     1.894327  0.795958   

                 Value At Risk            ...  \
bank                 Citigroup  JPMorgan  ...   
reporting_period                          ...   
2023Q1                0.102109  0.000000  ...   
2023Q2                0.000000  0.000000  ...   
2023Q3                0.000000 -0.270192  ...   
2023Q4               -0.014934  0.000000  ...   
2024Q1                0.317483  0.000000  ...   
2024Q2               -0.351328  0.000000  ...   
2024Q3                0.000000  0.000000  ...   
2024Q4                0.000000  0.000000  ...   
2025Q1               -0.126321  0.000000  ...   

                 RISK CATEGORY: Market and Volatility Risk            \
bank                                             Citigroup  JPMorgan   
reporting_period                                                       
2023Q1                                            1.088627  0.596191   
2023Q2                                            0.944383  0.265887   
2023Q3                                            0.971564  0.454212   
2023Q4                                            0.138879  1.134739   
2024Q1                                            0.898005  0.824595   
2024Q2                                            1.029296  1.099474   
2024Q3                                            0.909468  1.165339   
2024Q4                                            1.034389  0.898603   
2025Q1                                            1.050395  1.267111   

                 RISK CATEGORY: Operational Risk            \
bank                                   Citigroup  JPMorgan   
reporting_period                                             
2023Q1                                  0.290626  0.197963   
2023Q2              

In [107]:
df_topics_relevance_sentiment_quarter_plotting.columns = [', '.join(col).strip() for col in df_topics_relevance_sentiment_quarter_plotting.columns.values]
df_topics_relevance_sentiment_quarter_plotting.reset_index(inplace=True)
df_topics_relevance_sentiment_quarter_plotting

,reporting_period,"Macroeconomic Outlook, Citigroup","Macroeconomic Outlook, JPMorgan","Systemic Stability & Regulation, Citigroup","Systemic Stability & Regulation, JPMorgan","Business Transformation, Citigroup","Business Transformation, JPMorgan","Growth Plans, Citigroup","Growth Plans, JPMorgan","Value At Risk, Citigroup",...,"RISK CATEGORY: Market and Volatility Risk, Citigroup","RISK CATEGORY: Market and Volatility Risk, JPMorgan","RISK CATEGORY: Operational Risk, Citigroup","RISK CATEGORY: Operational Risk, JPMorgan","RISK CATEGORY: Profitability, Citigroup","RISK CATEGORY: Profitability, JPMorgan","RISK CATEGORY: Regulatory & Compliance Risk, Citigroup","RISK CATEGORY: Regulatory & Compliance Risk, JPMorgan","RISK CATEGORY: Strategic and Business Model Risk, Citigroup","RISK CATEGORY: Strategic and Business Model Risk, JPMorgan"
0,2023Q1,0.340327,-2.081631,0.220541,-0.218864,0.977047,0.000000,0.805073,-0.233276,0.102109,...,1.088627,0.596191,0.290626,0.197963,6.440624,7.776062,0.021869,-0.016603,0.846877,0.064535
1,2023Q2,0.035306,-0.051790,-0.175488,-0.987473,2.555608,0.008517,2.496304,1.821243,0.000000,...,0.944383,0.265887,0.143103,-0.093042,-0.559611,6.102575,-0.054363,-0.136230,3.020865,0.812704
2,2023Q3,-0.601743,-2.897782,-0.104564,-0.574525,2.746828,0.340243,1.441981,-0.012745,0.000000,...,0.971564,0.454212,0.329938,0.112852,5.572853,5.513535,-0.006511,-0.060268,1.475966,0.590608
3,2023Q4,0.317550,0.018854,0.119368,0.000000,3.621517,0.000000,2.876357,0.791051,-0.014934,...,0.138879,1.134739,0.442716,0.450345,1.902327,6.985511,0.020252,0.000000,3.087349,0.918343
4,2024Q1,-0.347859,0.116909,0.000000,0.000000,4.110513,0.000000,2.420880,0.846485,0.317483,...,0.898005,0.824595,0.412533,0.337987,4.783454,6.514100,0.038098,0.020544,2.513278,0.741798
5,2024Q2,-0.344308,-0.399786,0.302413,0.000000,4.337381,0.000000,2.398360,0.322455,-0.351328,...,1.029296,1.099474,-0.205209,0.277965,2.282180,6.242138,0.102170,0.000000,2.826735,0.523217
6,2024Q3,0.338124,0.346621,0.231008,0.298294,2.561336,0.000000,2.781485,0.629683,0.000000,...,0.909468,1.165339,0.382246,0.080667,5.660540,4.910978,0.053698,0.020470,1.630081,0.907798
7,2024Q4,0.200094,-0.176315,0.000000,0.000000,3.185312,0.382413,3.039439,0.389032,0.000000,...,1.034389,0.898603,0.336785,0.400199,6.238587,6.403574,0.030725,0.000000,1.670724,0.558520
8,2025Q1,0.258709,0.456509,0.207143,-0.241117,1.038792,0.285707,1.894327,0.795958,-0.126321,...,1.050395,1.267111,0.232925,0.411222,7.152725,7.012438,0.043918,-0.016065,1.067563,0.357393


In [111]:
# Generate a multiline chart for the net sentiment scores
risk_category = "Interest Rate Risk"
cols_to_plot = [col for col in risk_category_cols if risk_category in col]
legend_labels = [col.split(",")[-1].strip() for col in cols_to_plot]

fig_topics_relevance_sentiment_quarter = generate_multiline_chart(
	df=df_topics_relevance_sentiment_quarter_plotting,
	x_data_col='reporting_period',
	y_data_cols=cols_to_plot,
	x_title='Reporting Period',
	y_title='Net Sentiment Score (Normalized)',
	plot_title=f"Quarterly Net Sentiment Scores: {risk_category}",
	marker_size=5,
	line_legend_labels=legend_labels,
)

fig_topics_relevance_sentiment_quarter.show()

In [109]:
# Generate a multiline chart for the net sentiment scores
subtopic = "Deposit Trends and Pricing"
cols_to_plot = [col for col in subtopic_cols if subtopic in col]
legend_labels = [col.split(",")[-1].strip() for col in cols_to_plot]

fig_topics_relevance_sentiment_quarter = generate_multiline_chart(
	df=df_topics_relevance_sentiment_quarter_plotting,
	x_data_col='reporting_period',
	y_data_cols=cols_to_plot,
	x_title='Reporting Period',
	y_title='Net Sentiment Score (Normalized)',
	plot_title=f"Quarterly Net Sentiment Scores: {subtopic}",
	marker_size=5,
	line_legend_labels=legend_labels,
)

fig_topics_relevance_sentiment_quarter.show()